# Advent of Code 2017
## Evan Ackmann

### Day 0: The Junk I'll Probably Need For Everything
Just want to have a common place to deal with all the goofy A\* junk and any convenience functions that I require

In [ ]:
import math
import re
from hashlib import md5
from functools import partial, reduce
from itertools import permutations, combinations, tee
from collections import defaultdict
import time
import operator
import sys
from pathlib import Path
from typing import Tuple
from numba import jit
Point = Tuple[int,int]
Size = Tuple[int, int]

from requests import request
import json

def quadratic_solutions(a,b,c):
    solutions = []
    if a == 0:
        return None
    top_term = b**2 - 4 * a * c
    if top_term >= 0:
        return [(-b + math.sqrt(top_term))/(2*a),(-b - math.sqrt(top_term))/(2*a)]

    return None

def get_input(day, year):
    file_path = Path(f'day{day:02}_{year}.in')
    if not file_path.exists():
        print(f'Data for day {day} not available, downloading...')
          
        config = None
        with open('config.json') as f:
            config = json.load(f)
        url = f'https://adventofcode.com/{year}/day/{day}/input'
        res = request('GET',url,cookies=config)
        try:
            output = ''
            res.raise_for_status()
            with open(f'day{day:02}_{year}.in','wb') as f:
                for chunk in res.iter_content(chunk_size=128):
                    f.write(chunk)
        except:
            return []
    
    with file_path.open() as f:
        return f.read().splitlines()
    return []
def get_numbers(in_string):
    return [int(x) for x in re.findall('-?[0-9]+',in_string)]

def println(msg):
    print('\r{}'.format(msg),end='')
    #sys.stdout.write(msg+'\r')
    #sys.stdout.flush()
    

def first_or_default(it, default = None):
    try:
        return next(iter(it))
    except:
        return default

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)


def X(point):
    return point[0]


def Y(point):
    return point[1]

def Z(point):
    return point[2]

def neighbors4(point):
    x0 = X(point)
    y0 = Y(point)
    return [(x0 + 1, y0), (x0 - 1, y0), (x0, y0 + 1), (x0, y0 - 1)]

#this is good for flat tops, mostly, need to get the coordinate spaces right
def hex_neighbors(point):
    x0 = X(point)
    y0 = Y(point)
    z0 = Z(point)
    return [(x0+0,y0+1,z0-1),(x0+0,y0-1,z0+1),(x0+1,y0+0,z0-1),(x0+1,y0-1,z0+0),(x0-1,y0+1,z0+0),(x0-1,y0+0,z0+1)]

def neighbors8(point):
    x0 = X(point)
    y0 = Y(point)
    return [(x0 + 1, y0), (x0 - 1, y0), (x0, y0 + 1), (x0, y0 - 1), (x0 + 1, y0 + 1), (x0 - 1, y0 + 1), (x0 + 1, y0 - 1), (x0 - 1, y0 - 1)]


def euclidean_heuristic(start, end):
    x_d = abs(X(start) - X(end))
    y_d = abs(Y(start) - Y(end))
    return math.sqrt(x_d**2 + y_d**2)


def manhattan_heuristic(start, end):
    x_d = abs(X(start) - X(end))
    y_d = abs(Y(start) - Y(end))
    return x_d + y_d


def a_star(start, end, neighbors_fcn, heuristic_fcn, cost_fcn=None):
    closed_set = []
    open_set = [start]
    came_from = dict()
    g_score = dict()
    g_score[start] = 0
    f_score = dict()
    f_score[start] = heuristic_fcn(start, end)

    while len(open_set) > 0:
        current = min(
            open_set, key=lambda o: f_score[o] if o in f_score else math.inf)

        if current == end:
            final_path = [current]
            while current in came_from:
                current = came_from[current]
                final_path += [current]
            return list(reversed(final_path))

        open_set.remove(current)
        closed_set += [current]
        for neighbor in neighbors_fcn(current):
            if closed_set.count(neighbor) > 0:
                continue

            if open_set.count(neighbor) == 0:
                open_set += [neighbor]

            if cost_fcn:
                tentative_g_score = g_score[current] + \
                    cost_fcn(current, neighbor)
            else:
                tentative_g_score = g_score[current] + 1

            if neighbor in g_score and g_score[neighbor] <= tentative_g_score:
                continue  # already know better...

            came_from[neighbor] = current
            g_score[neighbor] = tentative_g_score
            f_score[neighbor] = g_score[neighbor] + \
                heuristic_fcn(neighbor, end)

    return []


def color(this_color, string):
    return "\033[" + this_color + "m" + string + "\033[0m"

def bold(msg):
    return u'\033[1m%s\033[0m' % msg

def underline(msg):
    return u'\033[4m%s\033[0m' % msg


def get_input_lines(in_file):
    with open(in_file) as f:
        return f.read().splitlines()
    return []


def primes_under(limit):
    a = [True] * limit
    a[0] = a[1] = False

    for (i, isprime) in enumerate(a):
        if isprime:
            yield i
            for n in range(i * i, limit, i):
                a[n] = False


def is_prime(number):
    for test_number in primes_under(math.ceil(math.sqrt(number))):
        if number % test_number == 0:
            return False
    return True


def prime_factors(number):
    factors = set()

    done = False

    working_number = number

    while not done:
        for num in primes_under(math.ceil(math.sqrt(working_number))):
            if working_number % num == 0:
                factors.add(num)
                working_number = working_number / num
                continue
        done = True
        factors.add(int(working_number))

    factors.discard(number)
    factors.discard(1)
    return factors

## Day 1: Inverse Captcha

The night before Christmas, one of Santa's Elves calls you in a panic. "The printer's broken! We can't print the Naughty or Nice List!" By the time you make it to sub-basement 17, there are only a few minutes until midnight. "We have a big problem," she says; "there must be almost fifty bugs in this system, but nothing else can print The List. Stand in this square, quick! There's no time to explain; if you can convince them to pay you in stars, you'll be able to--" She pulls a lever and the world goes blurry.

When your eyes can focus again, everything seems a lot more pixelated than before. She must have sent you inside the computer! You check the system clock: 25 milliseconds until midnight. With that much time, you should be able to collect all fifty stars by December 25th.

Collect stars by solving puzzles. Two puzzles will be made available on each day millisecond in the advent calendar; the second puzzle is unlocked when you complete the first. Each puzzle grants one star. Good luck!

You're standing in a room with "digitization quarantine" written in LEDs along one wall. The only door is locked, but it includes a small interface. "Restricted Area - Strictly No Digitized Users Allowed."

It goes on to explain that you may only leave by solving a captcha to prove you're not a human. Apparently, you only get one millisecond to solve the captcha: too fast for a normal human, but it feels like hours to you.

The captcha requires you to review a sequence of digits (your puzzle input) and find the sum of all digits that match the next digit in the list. The list is circular, so the digit after the last digit is the first digit in the list.

For example:

```
1122 produces a sum of 3 (1 + 2) because the first digit (1) matches the second digit and the third digit (2) matches the fourth digit.
1111 produces 4 because each digit (all 1) matches the next.
1234 produces 0 because no digit matches the next.
91212129 produces 9 because the only digit that matches the next one is the last digit, 9.
```

What is the solution to your captcha?

--- Part Two ---

You notice a progress bar that jumps to 50% completion. Apparently, the door isn't yet satisfied, but it did emit a star as encouragement. The instructions change:

Now, instead of considering the next digit, it wants you to consider the digit halfway around the circular list. That is, if your list contains 10 items, only include a digit in your sum if the digit 10/2 = 5 steps forward matches it. Fortunately, your list has an even number of elements.

For example:

```
1212 produces 6: the list contains 4 items, and all four digits match the digit 2 items ahead.
1221 produces 0, because every comparison is between a 1 and a 2.
123425 produces 4, because both 2s match each other, but no other digit has a match.
123123 produces 12.
12131415 produces 4.
```

What is the solution to your new captcha?

In [221]:
day1input = get_input(1,2017)[0]

def day1part1(inputstring):
    return sum([int(inputstring[x]) for x in range(len(inputstring)) if inputstring[x] == inputstring[(x+1)%len(inputstring)]])
def day1part2(inputstring):
    return sum([int(inputstring[x]) for x in range(len(inputstring)) if inputstring[x] == inputstring[(int(len(inputstring)/2) + x) % len(inputstring)]])

print("Part 1:",day1part1(day1input))
print("Part 2:",day1part2(day1input))


Part 1: 1141
Part 2: 950


## Day 2: Corruption Checksum

As you walk through the door, a glowing humanoid shape yells in your direction. "You there! Your state appears to be idle. Come help us repair the corruption in this spreadsheet - if we take another millisecond, we'll have to display an hourglass cursor!"

The spreadsheet consists of rows of apparently-random numbers. To make sure the recovery process is on the right track, they need you to calculate the spreadsheet's checksum. For each row, determine the difference between the largest value and the smallest value; the checksum is the sum of all of these differences.

For example, given the following spreadsheet:

```
5 1 9 5
7 5 3
2 4 6 8
```
```
The first row's largest and smallest values are 9 and 1, and their difference is 8.
The second row's largest and smallest values are 7 and 3, and their difference is 4.
The third row's difference is 6.
In this example, the spreadsheet's checksum would be 8 + 4 + 6 = 18.
```
What is the checksum for the spreadsheet in your puzzle input?

--- Part Two ---

"Great work; looks like we're on the right track after all. Here's a star for your effort." However, the program seems a little worried. Can programs be worried?

"Based on what we're seeing, it looks like all the User wanted is some information about the evenly divisible values in the spreadsheet. Unfortunately, none of us are equipped for that kind of calculation - most of us specialize in bitwise operations."

It sounds like the goal is to find the only two numbers in each row where one evenly divides the other - that is, where the result of the division operation is a whole number. They would like you to find those numbers on each line, divide them, and add up each line's result.

For example, given the following spreadsheet:

```
5 9 2 8
9 4 7 3
3 8 6 5
```
```
In the first row, the only two numbers that evenly divide are 8 and 2; the result of this division is 4.
In the second row, the two numbers are 9 and 3; the result is 3.
In the third row, the result is 2.
```
In this example, the sum of the results would be 4 + 3 + 2 = 9.

What is the sum of each row's result in your puzzle input?


In [222]:
day2input = get_input(2,2017)

def day2part1(data):
    return sum([max(val) - min(val) for val in data])

def day2part2(data):
    return sum([[x//y for x in row for y in row if x % y == 0 and x != y][0] for row in data])

data = [[int(y) for y in x.split()] for x in day2input]
print("Part 1:",day2part1(data))
print("Part 2:",day2part2(data))


Data for day 2 not available, downloading...
Part 1: 51139
Part 2: 272


## Day 3: Spiral Memory

You come across an experimental new kind of memory stored on an infinite two-dimensional grid.

Each square on the grid is allocated in a spiral pattern starting at a location marked 1 and then counting up while spiraling outward. For example, the first few squares are allocated like this:

```
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
```
While this is very space-efficient (no squares are skipped), requested data must be carried back to square 1 (the location of the only access port for this memory system) by programs that can only move up, down, left, or right. They always take the shortest path: the Manhattan Distance between the location of the data and square 1.

For example:

Data from square `1` is carried `0` steps, since it's at the access port.
Data from square `12` is carried `3` steps, such as: down, left, left.
Data from square `23` is carried only `2` steps: up twice.
Data from square `1024` must be carried `31` steps.
How many steps are required to carry the data from the square identified in your puzzle input all the way to the access port?

Your puzzle answer was 552.

--- Part Two ---

As a stress test on the system, the programs here clear the grid and then store the value 1 in square 1. Then, in the same allocation order as shown above, they store the sum of the values in all adjacent squares, including diagonals.

So, the first few squares' values are chosen as follows:

Square `1` starts with the value `1`.
Square `2` has only one adjacent filled square (with value 1), so it also stores `1`.
Square `3` has both of the above squares as neighbors and stores the sum of their values, `2`.
Square `4` has all three of the aforementioned squares as neighbors and stores the sum of their values, `4`.
Square `5` only has the first and fourth squares as neighbors, so it gets the value `5`.
Once a square is written, its value does not change. Therefore, the first few squares would receive the following values:

```
147  142  133  122   59
304    5    4    2   57
330   10    1    1   54
351   11   23   25   26
362  747  806--->   ...
```

What is the first value written that is larger than your puzzle input?

Your puzzle answer was 330785.


In [223]:
day3input = int(get_input(3,2017)[0])

def positions_for_square_with_size(num):
    # 1 -> (0,0)
    # 3 -> (0,0),(0,1),(0,2),(1,2),(2,2),(2,1),(2,0),(1,1)
    positions = []
    positions += [(0,y) for y in range(num)]
    positions += [(x,num-1) for x in range(1,num)]
    positions += [(num-1,num-1-y) for y in range(1,num)]
    positions += [(num-1-x,0) for x in range(1,num-1)]
    return positions

def next_spiral_row(num):
    #3 -> (1,0),(1,-1),(0,-1) (-1,-1),(-1,0),(-1,1)
    positions = [(num//2 - x, num//2 - y) for x,y in positions_for_square_with_size(num)]
    positions = positions[len(positions)//2 + 1:] + positions[:len(positions)//2 + 1]
    return positions
    
    
def max_dim(num):
    f = math.ceil(math.sqrt(num))
    if f % 2 == 0:
        f += 1
    return f


def day3part1(num):
    md = max_dim(num)
    positions = positions_for_square_with_size(md)
    positions = positions[len(positions)//2:] + positions[:len(positions)//2]
    pos = positions[md**2 - num]
    center = (md//2,md//2)
    return manhattan_heuristic(pos,center)

def spiral_path():
    current_rad = 3
    yield (0,0)
    while True:
        for p in next_spiral_row(current_rad):
                yield p
        current_rad += 2
    
def day3part2(num):
    grid = {(0,0):1}
    last_write = 1
    
    for p in spiral_path():
        if p == (0,0):
            continue
        grid[p] = sum([grid.get(n,0) for n in neighbors8(p)])

        if grid[p] > num:
            return grid[p]
            
print("Part 1:",day3part1(day3input))
print("Part 2:",day3part2(day3input))


Data for day 3 not available, downloading...
Part 1: 552
Part 2: 330785


## Day 4: High-Entropy Passphrases

A new system policy has been put in place that requires all accounts to use a passphrase instead of simply a password. A passphrase consists of a series of words (lowercase letters) separated by spaces.

To ensure security, a valid passphrase must contain no duplicate words.

For example:

aa bb cc dd ee is valid.
aa bb cc dd aa is not valid - the word aa appears more than once.
aa bb cc dd aaa is valid - aa and aaa count as different words.
The system's full passphrase list is available as your puzzle input. How many passphrases are valid?

Your puzzle answer was 451.

--- Part Two ---

For added security, yet another system policy has been put in place. Now, a valid passphrase must contain no two words that are anagrams of each other - that is, a passphrase is invalid if any word's letters can be rearranged to form any other word in the passphrase.

For example:

abcde fghij is a valid passphrase.
abcde xyz ecdab is not valid - the letters from the third word can be rearranged to form the first word.
a ab abc abd abf abj is a valid passphrase, because all letters need to be used when forming another word.
iiii oiii ooii oooi oooo is valid.
oiii ioii iioi iiio is not valid - any of these words can be rearranged to form any other word.
Under this new system policy, how many passphrases are valid?

Your puzzle answer was 223.

In [224]:
day4input = get_input(4,2017)

def letter_count(in_str):
    letters = dict()
    for c in in_str:
        if not c in letters:
            letters[c] = 1
        else:
            letters[c] += 1
    return letters

def day4part1(passwords):
    valid_passwords = [x for x in passwords if len(set(x.split())) == len(x.split())]
    return len(valid_passwords)
    
    
def day4part2(passwords):
    valid_passwords = []
    for password in passwords:
        is_valid = True
        for (a,b) in combinations(password.split(),2):
            if letter_count(a) == letter_count(b):
                is_valid = False
                break;
        if is_valid:
            valid_passwords += [password]
    return len(valid_passwords)
        
print("Part 1:", day4part1(day4input))
print("Part 2:",day4part2(day4input))


Data for day 4 not available, downloading...
Part 1: 451
Part 2: 223


## Day 5: A Maze of Twisty Trampolines, All Alike

An urgent interrupt arrives from the CPU: it's trapped in a maze of jump instructions, and it would like assistance from any programs with spare cycles to help find the exit.

The message includes a list of the offsets for each jump. Jumps are relative: -1 moves to the previous instruction, and 2 skips the next one. Start at the first instruction in the list. The goal is to follow the jumps until one leads outside the list.

In addition, these instructions are a little strange; after each jump, the offset of that instruction increases by 1. So, if you come across an offset of 3, you would move three instructions forward, but change it to a 4 for the next time it is encountered.

For example, consider the following list of jump offsets:

0
3
0
1
-3
Positive jumps ("forward") move downward; negative jumps move upward. For legibility in this example, these offset values will be written all on one line, with the current instruction marked in parentheses. The following steps would be taken before an exit is found:

(0) 3  0  1  -3  - before we have taken any steps.
(1) 3  0  1  -3  - jump with offset 0 (that is, don't jump at all). Fortunately, the instruction is then incremented to 1.
 2 (3) 0  1  -3  - step forward because of the instruction we just modified. The first instruction is incremented again, now to 2.
 2  4  0  1 (-3) - jump all the way to the end; leave a 4 behind.
 2 (4) 0  1  -2  - go back to where we just were; increment -3 to -2.
 2  5  0  1  -2  - jump 4 steps forward, escaping the maze.
In this example, the exit is reached in 5 steps.

How many steps does it take to reach the exit?

Your puzzle answer was 396086.

--- Part Two ---

Now, the jumps are even stranger: after each jump, if the offset was three or more, instead decrease it by 1. Otherwise, increase it by 1 as before.

Using this rule with the above example, the process now takes 10 steps, and the offset values after finding the exit are left as 2 3 2 3 -1.

How many steps does it now take to reach the exit?

Your puzzle answer was 28675390.

In [225]:
day5input = get_input(5,2017)

def day5part1(numbers):
    pc = 0
    count = 0
    while pc >= 0 and pc < len(numbers):
        count += 1
        new_pc = pc + numbers[pc]
        numbers[pc]+=1
        pc = new_pc
    return count

def day5part2(numbers):
    pc = 0
    count = 0
    while pc >= 0 and pc < len(numbers):
        count += 1
        new_pc = pc + numbers[pc]
        if(numbers[pc] >= 3):
            numbers[pc] -=1
        else:
            numbers[pc]+=1

        pc = new_pc
    return count

print("Part 1:", day5part1([int(x) for x in day5input]))
#print("Test Part 2:", day5part2([0,3,0,1,-3]))
print("Part 2:", day5part2([int(x) for x in day5input]))

Data for day 5 not available, downloading...
Part 1: 396086
Part 2: 28675390


## Day 6: Memory Reallocation

A debugger program here is having an issue: it is trying to repair a memory reallocation routine, but it keeps getting stuck in an infinite loop.

In this area, there are sixteen memory banks; each memory bank can hold any number of blocks. The goal of the reallocation routine is to balance the blocks between the memory banks.

The reallocation routine operates in cycles. In each cycle, it finds the memory bank with the most blocks (ties won by the lowest-numbered memory bank) and redistributes those blocks among the banks. To do this, it removes all of the blocks from the selected bank, then moves to the next (by index) memory bank and inserts one of the blocks. It continues doing this until it runs out of blocks; if it reaches the last memory bank, it wraps around to the first one.

The debugger would like to know how many redistributions can be done before a blocks-in-banks configuration is produced that has been seen before.

For example, imagine a scenario with only four memory banks:

The banks start with 0, 2, 7, and 0 blocks. The third bank has the most blocks, so it is chosen for redistribution.
Starting with the next bank (the fourth bank) and then continuing to the first bank, the second bank, and so on, the 7 blocks are spread out over the memory banks. The fourth, first, and second banks get two blocks each, and the third bank gets one back. The final result looks like this: 2 4 1 2.
Next, the second bank is chosen because it contains the most blocks (four). Because there are four memory banks, each gets one block. The result is: 3 1 2 3.
Now, there is a tie between the first and fourth memory banks, both of which have three blocks. The first bank wins the tie, and its three blocks are distributed evenly over the other three banks, leaving it with none: 0 2 3 4.
The fourth bank is chosen, and its four blocks are distributed such that each of the four banks receives one: 1 3 4 1.
The third bank is chosen, and the same thing happens: 2 4 1 2.
At this point, we've reached a state we've seen before: 2 4 1 2 was already seen. The infinite loop is detected after the fifth block redistribution cycle, and so the answer in this example is 5.

Given the initial block counts in your puzzle input, how many redistribution cycles must be completed before a configuration is produced that has been seen before?

Your puzzle answer was 4074.

--- Part Two ---

Out of curiosity, the debugger would also like to know the size of the loop: starting from a state that has already been seen, how many block redistribution cycles must be performed before that same state is seen again?

In the example above, 2 4 1 2 is seen again after four cycles, and so the answer in that example would be 4.

How many cycles are in the infinite loop that arises from the configuration in your puzzle input?

Your puzzle answer was 2793.

In [226]:
day6test = [0,2,7,0]
day6input = [int(x) for x in get_input(6,2017)[0].split()]

def get_cycle_history(initial_memory):
    memory = list(numbers)
    seen = []
    while memory not in seen:
        seen += [list(memory)]
        idx = memory.index(max(memory))
        val = memory[idx]
        memory[idx] = 0
        for i in range(val):
            memory[(idx + i + 1) % len(memory)] += 1
    return len(seen)

def day6part1(numbers):
    memory = list(numbers)
    seen = []
    while memory not in seen:
        seen += [list(memory)]
        idx = memory.index(max(memory))
        val = memory[idx]
        memory[idx] = 0
        for i in range(val):
            memory[(idx + i + 1) % len(memory)] += 1
    return len(seen)
        
def day6part2(numbers):
    memory = list(numbers)
    seen = []
    while memory not in seen:
        seen += [list(memory)]
        idx = memory.index(max(memory))
        val = memory[idx]
        memory[idx] = 0
        for i in range(val):
            memory[(idx + i + 1) % len(memory)] += 1
    return len(seen) - seen.index(memory)

print("Test 1:", day6part1(list(day6test)))
print("Part 1:", day6part1(list(day6input)))
print("Test 2:", day6part2(list(day6test)))
print("Part 2:", day6part2(list(day6input)))

Data for day 6 not available, downloading...
Test 1: 5
Part 1: 4074
Test 2: 4
Part 2: 2793


## Day 7: Recursive Circus

Wandering further through the circuits of the computer, you come upon a tower of programs that have gotten themselves into a bit of trouble. A recursive algorithm has gotten out of hand, and now they're balanced precariously in a large tower.

One program at the bottom supports the entire tower. It's holding a large disc, and on the disc are balanced several more sub-towers. At the bottom of these sub-towers, standing on the bottom disc, are other programs, each holding their own disc, and so on. At the very tops of these sub-sub-sub-...-towers, many programs stand simply keeping the disc below them balanced but with no disc of their own.

You offer to help, but first you need to understand the structure of these towers. You ask each program to yell out their name, their weight, and (if they're holding a disc) the names of the programs immediately above them balancing on that disc. You write this information down (your puzzle input). Unfortunately, in their panic, they don't do this in an orderly fashion; by the time you're done, you're not sure which program gave which information.

For example, if your list is the following:

pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)
...then you would be able to recreate the structure of the towers that looks like this:

                gyxo
              /     
         ugml - ebii
       /      \     
      |         jptl
      |        
      |         pbga
     /        /
tknk --- padx - havc
     \        \
      |         qoyq
      |             
      |         ktlj
       \      /     
         fwft - cntj
              \     
                xhth
In this example, tknk is at the bottom of the tower (the bottom program), and is holding up ugml, padx, and fwft. Those programs are, in turn, holding up other programs; in this example, none of those programs are holding up any other programs, and are all the tops of their own towers. (The actual tower balancing in front of you is much larger.)

Before you're ready to help them, you need to make sure your information is correct. What is the name of the bottom program?

Your puzzle answer was dgoocsw.

--- Part Two ---

The programs explain the situation: they can't get down. Rather, they could get down, if they weren't expending all of their energy trying to keep the tower balanced. Apparently, one program has the wrong weight, and until it's fixed, they're stuck here.

For any program holding a disc, each program standing on that disc forms a sub-tower. Each of those sub-towers are supposed to be the same weight, or the disc itself isn't balanced. The weight of a tower is the sum of the weights of the programs in that tower.

In the example above, this means that for ugml's disc to be balanced, gyxo, ebii, and jptl must all have the same weight, and they do: 61.

However, for tknk to be balanced, each of the programs standing on its disc and all programs above it must each match. This means that the following sums must all be the same:

ugml + (gyxo + ebii + jptl) = 68 + (61 + 61 + 61) = 251
padx + (pbga + havc + qoyq) = 45 + (66 + 66 + 66) = 243
fwft + (ktlj + cntj + xhth) = 72 + (57 + 57 + 57) = 243
As you can see, tknk's disc is unbalanced: ugml's stack is heavier than the other two. Even though the nodes above ugml are balanced, ugml itself is too heavy: it needs to be 8 units lighter for its stack to weigh 243 and keep the towers balanced. If this change were made, its weight would be 60.

Given that exactly one program is the wrong weight, what would its weight need to be to balance the entire tower?

Your puzzle answer was 1275.



In [227]:
day7test = '''pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)'''.splitlines()

day7input = get_input(7,2017)


def populate_node_map(node_list):
    def calc_weight_below(nodes, val):
        if len(nodes[val]['children']) == 0:
            return nodes[val]['weight']
        else:
            nodes[val]['weight_below'] = sum([calc_weight_below(nodes,x) for x in nodes[val]['children']])
            return nodes[val]['weight'] + nodes[val]['weight_below']
    
    
    nodes = dict()
    for line in node_list:
        tokens = [x.strip(' ').strip(',') for x in re.findall('\w+\W|\w+$|\(\d+\)',line) if x != '->']
        if tokens[0] not in nodes:
            nodes[tokens[0]] =  {'weight':int(tokens[1][1:-1]), 'children':[], 'parent': None, 'weight_below':0}
        nodes[tokens[0]]['weight'] = int(tokens[1][1:-1])
        if len(tokens) > 2:
            for child in tokens[2:]:
                if child not in nodes:
                    nodes[child] =  {'weight':int(tokens[1][1:-1]), 'children':[], 'parent': None, 'weight_below':0}
                nodes[child]['parent'] = tokens[0]
                nodes[tokens[0]]['children'] += [child]
    head = [k for k,v in nodes.items() if v['parent'] == None][0]
    calc_weight_below(nodes, head)
    return nodes

def day7part1(d7in):
    return [k for k,v in populate_node_map(d7in).items() if v['parent'] == None][0]
  
def day7part2(d7in):
    nodes = populate_node_map(d7in)
    head = [k for k,v in populate_node_map(d7in).items() if v['parent'] == None][0]
    current = head
    last_good = 0
    while True:
        #if they're all equal below it, then we've found it
        below_weights = [nodes[ch]['weight_below'] + nodes[ch]['weight'] for ch in nodes[current]['children']]
        if len(set(below_weights)) > 1:
            weights_present = list(set(below_weights))
            if below_weights.count(weights_present[0]) == 1:
                current = nodes[current]['children'][below_weights.index(weights_present[0])]
                last_good = weights_present[1]
            else:
                current = nodes[current]['children'][below_weights.index(weights_present[1])]
                last_good = weights_present[0]
        else:
            break#the last weight was our answer
        #balanced?
    return last_good - sum(below_weights)

assert(day7part1(list(day7test))=='tknk')
print("Part 1:", day7part1(list(day7input),))
assert(day7part2(list(day7test))==60)
print("Part 2:", day7part2(list(day7input)))

Data for day 7 not available, downloading...
Part 1: dgoocsw
Part 2: 1275


## Day 8: I Heard You Like Registers

You receive a signal directly from the CPU. Because of your recent assistance with jump instructions, it would like you to compute the result of a series of unusual register instructions.

Each instruction consists of several parts: the register to modify, whether to increase or decrease that register's value, the amount by which to increase or decrease it, and a condition. If the condition fails, skip the instruction without modifying the register. The registers all start at 0. The instructions look like this:
```
b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10
```
These instructions would be processed as follows:

Because a starts at 0, it is not greater than 1, and so b is not modified.
a is increased by 1 (to 1) because b is less than 5 (it is 0).
c is decreased by -10 (to 10) because a is now greater than or equal to 1 (it is 1).
c is increased by -20 (to -10) because c is equal to 10.
After this process, the largest value in any register is 1.

You might also encounter <= (less than or equal to) or != (not equal to). However, the CPU doesn't have the bandwidth to tell you what all the registers are named, and leaves that to you to determine.

What is the largest value in any register after completing the instructions in your puzzle input?

Your puzzle answer was 5221.

--- Part Two ---

To be safe, the CPU also needs to know the highest value held in any register during this process so that it can decide how much memory to allocate to these operations. For example, in the above instructions, the highest value ever held was 10 (in register c after the third instruction was evaluated).

Your puzzle answer was 7491.

In [228]:
day8test = '''b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10'''.splitlines()

day8in = get_input(8,2017)

def runday8(d8in):
    reg = defaultdict(int)
    peak = 0
    op= {'<':operator.lt, '>':operator.gt, '<=':operator.le, '>=':operator.ge, '==':operator.eq, '!=':operator.ne}
    for line in d8in:
        tok = line.split()
        amount = int(tok[2])
        if tok[1] == 'dec': amount *= -1
        if op[tok[5]](reg[tok[4]],int(tok[6])): reg[tok[0]] += amount
        peak = max(list(reg.values()) + [peak])
    return (max(list(reg.values())),peak)

def day8part1(d8in):
    return runday8(d8in)[0]

def day8part2(d8in):
    return runday8(d8in)[1]

assert(day8part1(day8test) == 1)
print("Part 1:",day8part1(day8in))
assert(day8part2(day8test) == 10)
print("Part 2:",day8part2(day8in))

Data for day 8 not available, downloading...
Part 1: 5221
Part 2: 7491


## Day 9: Stream Processing

A large stream blocks your path. According to the locals, it's not safe to cross the stream at the moment because it's full of garbage. You look down at the stream; rather than water, you discover that it's a stream of characters.

You sit for a while and record part of the stream (your puzzle input). The characters represent groups - sequences that begin with { and end with }. Within a group, there are zero or more other things, separated by commas: either another group or garbage. Since groups can contain other groups, a } only closes the most-recently-opened unclosed group - that is, they are nestable. Your puzzle input represents a single, large group which itself contains many smaller ones.

Sometimes, instead of a group, you will find garbage. Garbage begins with < and ends with >. Between those angle brackets, almost any character can appear, including { and }. Within garbage, < has no special meaning.

In a futile attempt to clean up the garbage, some program has canceled some of the characters within it using !: inside garbage, any character that comes after ! should be ignored, including <, >, and even another !.

You don't see any characters that deviate from these rules. Outside garbage, you only find well-formed groups, and garbage always terminates according to the rules above.

Here are some self-contained pieces of garbage:

<>, empty garbage.
<random characters>, garbage containing random characters.
<<<<>, because the extra < are ignored.
<{!>}>, because the first > is canceled.
<!!>, because the second ! is canceled, allowing the > to terminate the garbage.
<!!!>>, because the second ! and the first > are canceled.
<{o"i!a,<{i<a>, which ends at the first >.
Here are some examples of whole streams and the number of groups they contain:

{}, 1 group.
{{{}}}, 3 groups.
{{},{}}, also 3 groups.
{{{},{},{{}}}}, 6 groups.
{<{},{},{{}}>}, 1 group (which itself contains garbage).
{<a>,<a>,<a>,<a>}, 1 group.
{{<a>},{<a>},{<a>},{<a>}}, 5 groups.
{{<!>},{<!>},{<!>},{<a>}}, 2 groups (since all but the last > are canceled).
Your goal is to find the total score for all groups in your input. Each group is assigned a score which is one more than the score of the group that immediately contains it. (The outermost group gets a score of 1.)

{}, score of 1.
{{{}}}, score of 1 + 2 + 3 = 6.
{{},{}}, score of 1 + 2 + 2 = 5.
{{{},{},{{}}}}, score of 1 + 2 + 3 + 3 + 3 + 4 = 16.
{<a>,<a>,<a>,<a>}, score of 1.
{{<ab>},{<ab>},{<ab>},{<ab>}}, score of 1 + 2 + 2 + 2 + 2 = 9.
{{<!!>},{<!!>},{<!!>},{<!!>}}, score of 1 + 2 + 2 + 2 + 2 = 9.
{{<a!>},{<a!>},{<a!>},{<ab>}}, score of 1 + 2 = 3.
What is the total score for all groups in your input?

Your puzzle answer was 23588.

--- Part Two ---

Now, you're ready to remove the garbage.

To prove you've removed it, you need to count all of the characters within the garbage. The leading and trailing < and > don't count, nor do any canceled characters or the ! doing the canceling.

<>, 0 characters.
<random characters>, 17 characters.
<<<<>, 3 characters.
<{!>}>, 2 characters.
<!!>, 0 characters.
<!!!>>, 0 characters.
<{o"i!a,<{i<a>, 10 characters.
How many non-canceled characters are within the garbage in your puzzle input?

Your puzzle answer was 10045.


In [229]:
day9test1 = '{{{{<asd>},{<!asd>}}}}'

day9in = get_input(9,2017)

def doday9(d9in):
    count = 0
    last_value = 0
    garbage_count = 0
    i = 0
    in_garbage = False
    while i < len(d9in):
        if d9in[i] == '{' and not in_garbage:
            last_value += 1
            count += last_value
        elif d9in[i] == '}' and not in_garbage:
            last_value -= 1
        elif d9in[i] == '<' and not in_garbage:
            in_garbage = True
        elif d9in[i] == '>' and in_garbage:
            in_garbage = False
        elif d9in[i] == '!' and in_garbage:
            i += 1
        elif in_garbage:
            garbage_count += 1
        
        i+=1
    return (count, garbage_count)
def day9part1(d9in):
    return doday9(d9in)[0]

def day9part2(d9in):
    return doday9(d9in)[1]

assert(day9part1(day9test1) == 14)
print("Part 1:",day9part1(day9in[0]))
assert(day9part2(day9test1) == 5)
print("Part 2:",day9part2(day9in[0]))

Data for day 9 not available, downloading...
Part 1: 23588
Part 2: 10045


## Day 10: Knot Hash

You come across some programs that are trying to implement a software emulation of a hash based on knot-tying. The hash these programs are implementing isn't very strong, but you decide to help them anyway. You make a mental note to remind the Elves later not to invent their own cryptographic functions.

This hash function simulates tying a knot in a circle of string with 256 marks on it. Based on the input to be hashed, the function repeatedly selects a span of string, brings the ends together, and gives the span a half-twist to reverse the order of the marks within it. After doing this many times, the order of the marks is used to build the resulting hash.
```
  4--5   pinch   4  5           4   1
 /    \  5,0,1  / \/ \  twist  / \ / \
3      0  -->  3      0  -->  3   X   0
 \    /         \ /\ /         \ / \ /
  2--1           2  1           2   5
  ```
To achieve this, begin with a list of numbers from 0 to 255, a current position which begins at 0 (the first element in the list), a skip size (which starts at 0), and a sequence of lengths (your puzzle input). Then, for each length:

Reverse the order of that length of elements in the list, starting with the element at the current position.
Move the current position forward by that length plus the skip size.
Increase the skip size by one.
The list is circular; if the current position and the length try to reverse elements beyond the end of the list, the operation reverses using as many extra elements as it needs from the front of the list. If the current position moves past the end of the list, it wraps around to the front. Lengths larger than the size of the list are invalid.

Here's an example using a smaller list:

Suppose we instead only had a circular list containing five elements, 0, 1, 2, 3, 4, and were given input lengths of 3, 4, 1, 5.

The list begins as [0] 1 2 3 4 (where square brackets indicate the current position).
The first length, 3, selects ([0] 1 2) 3 4 (where parentheses indicate the sublist to be reversed).
After reversing that section (0 1 2 into 2 1 0), we get ([2] 1 0) 3 4.
Then, the current position moves forward by the length, 3, plus the skip size, 0: 2 1 0 [3] 4. Finally, the skip size increases to 1.
The second length, 4, selects a section which wraps: 2 1) 0 ([3] 4.
The sublist 3 4 2 1 is reversed to form 1 2 4 3: 4 3) 0 ([1] 2.
The current position moves forward by the length plus the skip size, a total of 5, causing it not to move because it wraps around: 4 3 0 [1] 2. The skip size increases to 2.
The third length, 1, selects a sublist of a single element, and so reversing it has no effect.
The current position moves forward by the length (1) plus the skip size (2): 4 [3] 0 1 2. The skip size increases to 3.
The fourth length, 5, selects every element starting with the second: 4) ([3] 0 1 2. Reversing this sublist (3 0 1 2 4 into 4 2 1 0 3) produces: 3) ([4] 2 1 0.
Finally, the current position moves forward by 8: 3 4 2 1 [0]. The skip size increases to 4.
In this example, the first two numbers in the list end up being 3 and 4; to check the process, you can multiply them together to produce 12.

However, you should instead use the standard list size of 256 (with values 0 to 255) and the sequence of lengths in your puzzle input. Once this process is complete, what is the result of multiplying the first two numbers in the list?

Your puzzle answer was 38628.

--- Part Two ---

The logic you've constructed forms a single round of the Knot Hash algorithm; running the full thing requires many of these rounds. Some input and output processing is also required.

First, from now on, your input should be taken not as a list of numbers, but as a string of bytes instead. Unless otherwise specified, convert characters to bytes using their ASCII codes. This will allow you to handle arbitrary ASCII strings, and it also ensures that your input lengths are never larger than 255. For example, if you are given 1,2,3, you should convert it to the ASCII codes for each character: 49,44,50,44,51.

Once you have determined the sequence of lengths to use, add the following lengths to the end of the sequence: 17, 31, 73, 47, 23. For example, if you are given 1,2,3, your final sequence of lengths should be 49,44,50,44,51,17,31,73,47,23 (the ASCII codes from the input string combined with the standard length suffix values).

Second, instead of merely running one round like you did above, run a total of 64 rounds, using the same length sequence in each round. The current position and skip size should be preserved between rounds. For example, if the previous example was your first round, you would start your second round with the same length sequence (3, 4, 1, 5, 17, 31, 73, 47, 23, now assuming they came from ASCII codes and include the suffix), but start with the previous round's current position (4) and skip size (4).

Once the rounds are complete, you will be left with the numbers from 0 to 255 in some order, called the sparse hash. Your next task is to reduce these to a list of only 16 numbers called the dense hash. To do this, use numeric bitwise XOR to combine each consecutive block of 16 numbers in the sparse hash (there are 16 such blocks in a list of 256 numbers). So, the first element in the dense hash is the first sixteen elements of the sparse hash XOR'd together, the second element in the dense hash is the second sixteen elements of the sparse hash XOR'd together, etc.

For example, if the first sixteen elements of your sparse hash are as shown below, and the XOR operator is ^, you would calculate the first output number like this:

65 ^ 27 ^ 9 ^ 1 ^ 4 ^ 3 ^ 40 ^ 50 ^ 91 ^ 7 ^ 6 ^ 0 ^ 2 ^ 5 ^ 68 ^ 22 = 64
Perform this operation on each of the sixteen blocks of sixteen numbers in your sparse hash to determine the sixteen numbers in your dense hash.

Finally, the standard way to represent a Knot Hash is as a single hexadecimal string; the final output is the dense hash in hexadecimal notation. Because each number in your dense hash will be between 0 and 255 (inclusive), always represent each number as two hexadecimal digits (including a leading zero as necessary). So, if your first three numbers are 64, 7, 255, they correspond to the hexadecimal numbers 40, 07, ff, and so the first six characters of the hash would be 4007ff. Because every Knot Hash is sixteen such numbers, the hexadecimal representation is always 32 hexadecimal digits (0-f) long.

Here are some example hashes:

The empty string becomes a2582a3a0e66e6e86e3812dcb672a272.
AoC 2017 becomes 33efeb34ea91902bb2f59c9920caa6cd.
1,2,3 becomes 3efbe78a8d82f29979031a4aa0b16a9d.
1,2,4 becomes 63960835bcdc130f0b66d7ff4f6a5a8e.
Treating your puzzle input as a string of ASCII characters, what is the Knot Hash of your puzzle input? Ignore any leading or trailing whitespace you might encounter.

Your puzzle answer was e1462100a34221a7f0906da15c1c979a.

In [230]:
day10test_1 = [int(x) for x in '3,4,1,5'.split(',')]

day10in = get_input(10,2017)[0]

day10in_1 = [int(x) for x in day10in.split(',')]
day10test_2 = [ord(x) for x in '3,4,1,5']
day10in_2 = [ord(x) for x in day10in]

def do_day_10_1(d10in,npositions):
    positions = list(range(npositions))
    current_pos = 0
    skip_size = 0
    for length in d10in:
        #reverse
        for i in range(length//2):
            a = (current_pos + i) % len(positions)
            b = (current_pos + (length-1) - i) % len(positions)
            positions[a], positions[b] = positions[b], positions[a]
        
        #move
        current_pos += (skip_size + length) % len(positions)
        
        #increase skip size
        skip_size += 1
        pass

    return positions[0]*positions[1]
 
def knot_hash(a_string):
    return do_day_10_2([ord(x) for x in a_string], 256)
    
def do_day_10_2(d10in,npositions):
    positions = list(range(npositions))
    d10in += [17, 31, 73, 47, 23]
    current_pos = 0
    skip_size = 0
    for _  in range(64):
        for length in d10in:
            #reverse
            for i in range(length//2):
                a = (current_pos + i) % len(positions)
                b = (current_pos + (length-1) - i) % len(positions)
                positions[a], positions[b] = positions[b], positions[a]

            #move
            current_pos += (skip_size + length) % len(positions)

            #increase skip size
            skip_size += 1
            pass
    res = [0] * 16
    for a in range(16):
        res[a] =    positions[(a * 16) + 0] ^ \
                    positions[(a * 16) + 1] ^ \
                    positions[(a * 16) + 2] ^ \
                    positions[(a * 16) + 3] ^ \
                    positions[(a * 16) + 4] ^ \
                    positions[(a * 16) + 5] ^ \
                    positions[(a * 16) + 6] ^ \
                    positions[(a * 16) + 7] ^ \
                    positions[(a * 16) + 8] ^ \
                    positions[(a * 16) + 9] ^ \
                    positions[(a * 16) + 10] ^ \
                    positions[(a * 16) + 11] ^ \
                    positions[(a * 16) + 12] ^ \
                    positions[(a * 16) + 13] ^ \
                    positions[(a * 16) + 14] ^ \
                    positions[(a * 16) + 15]
    return ''.join(['{:02x}'.format(x) for x in res])
 
    
assert(do_day_10_1(day10test_1,5) == 12)
print("Part 1:",do_day_10_1(day10in_1,256))
#assert(do_day_10_2(day10test_2,5) == None)
print("Part 2:",do_day_10_2(day10in_2,256))
                          

Data for day 10 not available, downloading...
Part 1: 38628
Part 2: e1462100a34221a7f0906da15c1c979a


## Day 11: Hex Ed

Crossing the bridge, you've barely reached the other side of the stream when a program comes up to you, clearly in distress. "It's my child process," she says, "he's gotten lost in an infinite grid!"

Fortunately for her, you have plenty of experience with infinite grids.

Unfortunately for you, it's a hex grid.

The hexagons ("hexes") in this grid are aligned such that adjacent hexes can be found to the north, northeast, southeast, south, southwest, and northwest:

```
  \ n  /
nw +--+ ne
  /    \ 
-+      +-
  \    /
sw +--+ se
  / s  \
  ```
  
You have the path the child process took. Starting where he started, you need to determine the fewest number of steps required to reach him. (A "step" means to move from the hex you are in to any adjacent hex.)

For example:

ne,ne,ne is 3 steps away.
ne,ne,sw,sw is 0 steps away (back where you started).
ne,ne,s,s is 2 steps away (se,se).
se,sw,se,sw,sw is 3 steps away (s,s,sw).
Your puzzle answer was 812.

--- Part Two ---

How many steps away is the furthest he ever got from his starting position?

Your puzzle answer was 1603.

In [231]:
day11in = get_input(11,2017)[0].split(',')

def day11part1(d11in):
    dirs = {'n':(0,1,-1),'s':(0,-1,1),'ne':(1,0,-1),'se':(1,-1,0),'nw':(-1,1,0),'sw':(-1,0,1)}
    pos = (0,0,0)
    dists = []
    def distance(a,b):
        return max(abs(a[0]-b[0]),abs(a[1]-b[1]),abs(a[2]-b[2]))
    for d in d11in:
        pos = (pos[0] + dirs[d][0], pos[1] + dirs[d][1], pos[2] + dirs[d][2])  
        dists+=[distance(pos,(0,0,0))]
        
    def distance(a,b):
        return max(abs(a[0]-b[0]),abs(a[1]-b[1]),abs(a[2]-b[2]))
    print(distance(pos,(0,0,0)),max(dists))
    
day11part1(day11in)
    

Data for day 11 not available, downloading...
812 1603


## Day 12: Digital Plumber

Walking along the memory banks of the stream, you find a small village that is experiencing a little confusion: some programs can't communicate with each other.

Programs in this village communicate using a fixed system of pipes. Messages are passed between programs using these pipes, but most programs aren't connected to each other directly. Instead, programs pass messages between each other until the message reaches the intended recipient.

For some reason, though, some of these messages aren't ever reaching their intended recipient, and the programs suspect that some pipes are missing. They would like you to investigate.

You walk through the village and record the ID of each program and the IDs with which it can communicate directly (your puzzle input). Each program has one or more programs with which it can communicate, and these pipes are bidirectional; if 8 says it can communicate with 11, then 11 will say it can communicate with 8.

You need to figure out how many programs are in the group that contains program ID 0.

For example, suppose you go door-to-door like a travelling salesman and record the following list:
```
0 <-> 2
1 <-> 1
2 <-> 0, 3, 4
3 <-> 2, 4
4 <-> 2, 3, 6
5 <-> 6
6 <-> 4, 5
```
In this example, the following programs are in the group that contains program ID 0:
```
Program 0 by definition.
Program 2, directly connected to program 0.
Program 3 via program 2.
Program 4 via program 2.
Program 5 via programs 6, then 4, then 2.
Program 6 via programs 4, then 2.
```
Therefore, a total of 6 programs are in this group; all but program 1, which has a pipe that connects it to itself.

How many programs are in the group that contains program ID 0?

Your puzzle answer was 115.

--- Part Two ---

There are more programs than just the ones in the group containing program ID 0. The rest of them have no way of reaching that group, and still might have no way of reaching each other.

A group is a collection of programs that can all communicate via pipes either directly or indirectly. The programs you identified just a moment ago are all part of the same group. Now, they would like you to determine the total number of groups.

In the example above, there were 2 groups: one consisting of programs 0,2,3,4,5,6, and the other consisting solely of program 1.

How many groups are there in total?

Your puzzle answer was 221.

In [232]:
day12in = get_input(12,2017)

def doday12(d12in):
    mapping = defaultdict(set)
    for line in d12in:
        finds = [int(x) for x in re.findall('\d+',line)]
        for reverse in finds[1:]:
            mapping[finds[0]].add(reverse)
            mapping[reverse].add(finds[0])
    in_group = set([0])
    to_search = [0]
    while len(to_search) > 0:
        current = to_search.pop()
        for item in mapping[current]:
            if item == current:
                continue
            if item not in in_group:
                in_group.add(item)
                if item not in to_search:
                    to_search += [item]
    
    part1 = len(in_group)
    num_groups = 0
    while len(mapping) > 0:
        to_search = [next(iter(mapping))]
        in_group = set([to_search[0]])
        num_groups += 1
        while len(to_search) > 0:
            current = to_search.pop()
            for item in mapping[current]:
                if item == current:
                    continue
                if item not in in_group:
                    in_group.add(item)
                    if item not in to_search:
                        to_search += [item]
        for i in in_group:
            del mapping[i]
    
    return (part1, num_groups)

res = doday12(day12in)
print("Part 1:",res[0])
print("Part 2:",res[1])



Data for day 12 not available, downloading...
Part 1: 115
Part 2: 221


## Day 13: Packet Scanners

You need to cross a vast firewall. The firewall consists of several layers, each with a security scanner that moves back and forth across the layer. To succeed, you must not be detected by a scanner.

By studying the firewall briefly, you are able to record (in your puzzle input) the depth of each layer and the range of the scanning area for the scanner within it, written as depth: range. Each layer has a thickness of exactly 1. A layer at depth 0 begins immediately inside the firewall; a layer at depth 1 would start immediately after that.

For example, suppose you've recorded the following:
```
0: 3
1: 2
4: 4
6: 4
```
This means that there is a layer immediately inside the firewall (with range 3), a second layer immediately after that (with range 2), a third layer which begins at depth 4 (with range 4), and a fourth layer which begins at depth 6 (also with range 4). Visually, it might look like this:
```
 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]
```
Within each layer, a security scanner moves back and forth within its range. Each security scanner starts at the top and moves down until it reaches the bottom, then moves up until it reaches the top, and repeats. A security scanner takes one picosecond to move one step. Drawing scanners as S, the first few picoseconds look like this:


Picosecond 0:
```
 0   1   2   3   4   5   6
[S] [S] ... ... [S] ... [S]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]
```
Picosecond 1:
```
 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]
```
Picosecond 2:
```
 0   1   2   3   4   5   6
[ ] [S] ... ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[S]             [S]     [S]
                [ ]     [ ]
```
Picosecond 3:
```
 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] ... [ ]
[S] [S]         [ ]     [ ]
[ ]             [ ]     [ ]
                [S]     [S]
                ```
Your plan is to hitch a ride on a packet about to move through the firewall. The packet will travel along the top of each layer, and it moves at one layer per picosecond. Each picosecond, the packet moves one layer forward (its first move takes it into layer 0), and then the scanners move one step. If there is a scanner at the top of the layer as your packet enters it, you are caught. (If a scanner moves into the top of its layer while you are there, you are not caught: it doesn't have time to notice you before you leave.) If you were to do this in the configuration above, marking your current position with parentheses, your passage through the firewall would look like this:

Initial state:
```
 0   1   2   3   4   5   6
[S] [S] ... ... [S] ... [S]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]
```
Picosecond 0:
```
 0   1   2   3   4   5   6
(S) [S] ... ... [S] ... [S]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]

 0   1   2   3   4   5   6
( ) [ ] ... ... [ ] ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]
```

Picosecond 1:
```
 0   1   2   3   4   5   6
[ ] ( ) ... ... [ ] ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] (S) ... ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[S]             [S]     [S]
                [ ]     [ ]

```
Picosecond 2:
```
 0   1   2   3   4   5   6
[ ] [S] (.) ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[S]             [S]     [S]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [ ] (.) ... [ ] ... [ ]
[S] [S]         [ ]     [ ]
[ ]             [ ]     [ ]
                [S]     [S]

```
Picosecond 3:
```
 0   1   2   3   4   5   6
[ ] [ ] ... (.) [ ] ... [ ]
[S] [S]         [ ]     [ ]
[ ]             [ ]     [ ]
                [S]     [S]

 0   1   2   3   4   5   6
[S] [S] ... (.) [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[ ]             [S]     [S]
                [ ]     [ ]

```
Picosecond 4:
```
 0   1   2   3   4   5   6
[S] [S] ... ... ( ) ... [ ]
[ ] [ ]         [ ]     [ ]
[ ]             [S]     [S]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [ ] ... ... ( ) ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]

```
Picosecond 5:
```
 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] (.) [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [S] ... ... [S] (.) [S]
[ ] [ ]         [ ]     [ ]
[S]             [ ]     [ ]
                [ ]     [ ]

```
Picosecond 6:
```
 0   1   2   3   4   5   6
[ ] [S] ... ... [S] ... (S)
[ ] [ ]         [ ]     [ ]
[S]             [ ]     [ ]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] ... ( )
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]
                ```
In this situation, you are caught in layers 0 and 6, because your packet entered the layer when its scanner was at the top when you entered it. You are not caught in layer 1, since the scanner moved into the top of the layer once you were already there.

The severity of getting caught on a layer is equal to its depth multiplied by its range. (Ignore layers in which you do not get caught.) The severity of the whole trip is the sum of these values. In the example above, the trip severity is 0*3 + 6*4 = 24.

Given the details of the firewall you've recorded, if you leave immediately, what is the severity of your whole trip?

Your puzzle answer was 2264.

--- Part Two ---

Now, you need to pass through the firewall without being caught - easier said than done.

You can't control the speed of the packet, but you can delay it any number of picoseconds. For each picosecond you delay the packet before beginning your trip, all security scanners move one step. You're not in the firewall during this time; you don't enter layer 0 until you stop delaying the packet.

In the example above, if you delay 10 picoseconds (picoseconds 0 - 9), you won't get caught:

State after delaying:
```
 0   1   2   3   4   5   6
[ ] [S] ... ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[S]             [S]     [S]
                [ ]     [ ]
```
Picosecond 10:
```
 0   1   2   3   4   5   6
( ) [S] ... ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[S]             [S]     [S]
                [ ]     [ ]

 0   1   2   3   4   5   6
( ) [ ] ... ... [ ] ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]

```
Picosecond 11:
```
 0   1   2   3   4   5   6
[ ] ( ) ... ... [ ] ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]

 0   1   2   3   4   5   6
[S] (S) ... ... [S] ... [S]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]

```
Picosecond 12:
```
 0   1   2   3   4   5   6
[S] [S] (.) ... [S] ... [S]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [ ] (.) ... [ ] ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]
```

Picosecond 13:
```
 0   1   2   3   4   5   6
[ ] [ ] ... (.) [ ] ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [S] ... (.) [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[S]             [S]     [S]
                [ ]     [ ]
```

Picosecond 14:
```
 0   1   2   3   4   5   6
[ ] [S] ... ... ( ) ... [ ]
[ ] [ ]         [ ]     [ ]
[S]             [S]     [S]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [ ] ... ... ( ) ... [ ]
[S] [S]         [ ]     [ ]
[ ]             [ ]     [ ]
                [S]     [S]

```
Picosecond 15:
```
 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] (.) [ ]
[S] [S]         [ ]     [ ]
[ ]             [ ]     [ ]
                [S]     [S]

 0   1   2   3   4   5   6
[S] [S] ... ... [ ] (.) [ ]
[ ] [ ]         [ ]     [ ]
[ ]             [S]     [S]
                [ ]     [ ]
```

Picosecond 16:
```
 0   1   2   3   4   5   6
[S] [S] ... ... [ ] ... ( )
[ ] [ ]         [ ]     [ ]
[ ]             [S]     [S]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] ... ( )
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]
                ```
Because all smaller delays would get you caught, the fewest number of picoseconds you would need to delay to get through safely is 10.

What is the fewest number of picoseconds that you need to delay the packet to pass through the firewall without being caught?

Your puzzle answer was *3875838*.

In [233]:
day13test = '''0: 3
1: 2
4: 4
6: 4'''.splitlines()
day13in = get_input(13,2017)#('day13.in')

def doday13(d13in):
    scanner_model = dict()
    for line in d13in:
        elements = re.findall('\d+',line)
        scanner_model[int(elements[0])] ={'size':int(elements[1]),'scan_positions':list(range(int(elements[1]))) +list(range(int(elements[1])-2,0,-1))}
   
    def get_severity_with_start_time(model, start):
        severity = 0
        for k in model.keys():
            if model[k]['scan_positions'][(start + k)  % len(model[k]['scan_positions'])] == 0:
                severity += k * model[k]['size']
        return severity
   
    def will_be_caught_with_start_time(model, start):
        for k in model.keys():
            if model[k]['scan_positions'][(start + k)  % len(model[k]['scan_positions'])] == 0:
                return True
        return False
    first = 0
    
    while will_be_caught_with_start_time(scanner_model,first):
        first += 1

    return get_severity_with_start_time(scanner_model,0), first

test = doday13(day13test)
print("Test 1:",test[0])
print("Test 2:",test[1])
res = doday13(day13in)
print("Part 1:",res[0])
print("Part 2:",res[1])


Data for day 13 not available, downloading...
Test 1: 24
Test 2: 10
Part 1: 2264
Part 2: 3875838


## Day 14: Disk Defragmentation

Suddenly, a scheduled job activates the system's disk defragmenter. Were the situation different, you might sit and watch it for a while, but today, you just don't have that kind of time. It's soaking up valuable system resources that are needed elsewhere, and so the only option is to help it finish its task as soon as possible.

The disk in question consists of a 128x128 grid; each square of the grid is either free or used. On this disk, the state of the grid is tracked by the bits in a sequence of knot hashes.

A total of 128 knot hashes are calculated, each corresponding to a single row in the grid; each hash contains 128 bits which correspond to individual grid squares. Each bit of a hash indicates whether that square is free (0) or used (1).

The hash inputs are a key string (your puzzle input), a dash, and a number from 0 to 127 corresponding to the row. For example, if your key string were flqrgnkx, then the first row would be given by the bits of the knot hash of flqrgnkx-0, the second row from the bits of the knot hash of flqrgnkx-1, and so on until the last row, flqrgnkx-127.

The output of a knot hash is traditionally represented by 32 hexadecimal digits; each of these digits correspond to 4 bits, for a total of 4 * 32 = 128 bits. To convert to bits, turn each hexadecimal digit to its equivalent binary value, high-bit first: 0 becomes 0000, 1 becomes 0001, e becomes 1110, f becomes 1111, and so on; a hash that begins with a0c2017... in hexadecimal would begin with 10100000110000100000000101110000... in binary.

Continuing this process, the first 8 rows and columns for key flqrgnkx appear as follows, using # to denote used squares, and . to denote free ones:

##.#.#..-->
.#.#.#.#   
....#.#.   
#.#.##.#   
.##.#...   
##..#..#   
.#...#..   
##.#.##.-->
|      |   
V      V   
In this example, 8108 squares are used across the entire 128x128 grid.

Given your actual key string, how many squares are used?

Your puzzle answer was 8226.

--- Part Two ---
Now, all the defragmenter needs to know is the number of regions. A region is a group of used squares that are all adjacent, not including diagonals. Every used square is in exactly one region: lone used squares form their own isolated regions, while several adjacent squares all count as a single region.

In the example above, the following nine regions are visible, each marked with a distinct digit:

11.2.3..-->
.1.2.3.4   
....5.6.   
7.8.55.9   
.88.5...   
88..5..8   
.8...8..   
88.8.88.-->
|      |   
V      V   
Of particular interest is the region marked 8; while it does not appear contiguous in this small view, all of the squares marked 8 are connected when considering the whole 128x128 grid. In total, in this example, 1242 regions are present.

How many regions are present given your key string?

Your puzzle answer was 1128.

In [234]:
day14input = get_input(14,2017)[0]
day14test = 'flqrgnkx'

H_TO_B= {
    '0':'0000',
    '1':'0001',
}

def doday14(d14in):
    hashes = dict()
    for y in range(128):
        to_hash = d14in + '-{}'.format(y)
        hashed_value = knot_hash(to_hash)
        
        hashstring = ''.join([format(int(x,16), '04b') for x in hashed_value])
        for x,v in enumerate(hashstring):
            hashes[(x,y)] = ['.','#'][v=='1']
    

    ones = sum([1 for k,v in hashes.items() if v == '#'])
    
    for y in range(128):
        for x in range(128):
            print(hashes[(x,y)],end='')
        print("")
    
    def find_region(grid):
        for k,v in grid.items():
            if v == '#':

                return k
        return None
    
    current_region = 0
    point_to_start_next_region = find_region(hashes)
    while point_to_start_next_region:
        to_search = [point_to_start_next_region]
        while len(to_search) > 0:
            current = to_search.pop()
            if current in hashes and hashes[current] == '#':
                hashes[current] = '{}'.format(current_region)
                to_search += neighbors4(current)
        current_region += 1
        point_to_start_next_region = find_region(hashes)
        
    return ones, current_region

res = doday14(day14input)
print("Part 1:",res[0])
print("Part 2:",res[1])


Data for day 14 not available, downloading...
##..#.#.##.#...#.#....#########.##.###.###.##.#..#.##..####..#..#....#.#.###.#.##.....###......##..###..####.##.#..#.#.#...#...#
...#.....#.#.#.#..#....#..#...##...#..##.####....#....#..##.........###.#...####.#...###.##.###.....#..##.#..#.####.....###.#.#.
###.###....##.#.#...#####...#.##.#.##.#.#.#...###...#.#..##..#########.####..#.#.#####.#.#.##.#.##.....##...####.....##.#..#..##
##.#.#..#.#.#.......####..##.#..##..##..####...###.#..#.#.#####....#.###..##.#.#..#####.###.##.#..####.##....#..#..##..###...###
#..#..#...##.##......#.####..#.###....######...#..#.#.#########..##..##.#.##.....####....#.....#.##.####.#..#.#.##..###.#.#..###
##......####...##..#..###..####...###..#.###.........#....##..##.....#.##.##..#..########..##..##...#.#...#..#..#.#..###...##.#.
#.###.###.##.###..###.#.....###.##...###..##.#####..##.###...#..##.#......##.#..###.#..#..#.#..#..##....#.#...#.#..########.#...
##..###.#..#.######.#..#..##.....###..#...##.#.#...

.##.#.#.#####.#..#.#####.##.....#...##.###.#....#.#####.##..#.###...##...###.#.##....#.####.####.#..#.##..#...###.....#.#######.
##....##..##..####.##.#...##...#....#..###.###..##.####.##.#####.##.##.###..#....#...#..###..#.##.#...#.##.#...##.#..##....####.
###..##.#....#.######..#....#######.#.###.#....#..#....#.#.##.##.##.##.###..###..#..##.#..##############.##.#.#.##.#.#....##.#.#
#.#...##..##..##..###.#.#..####.#......###...##.###.........##...##.....#.#...###...#..#.###.##..##.###.....#..#..###...######..
.##.#.#.##.###.##.#.#####.#.####.#.#....##.#####.##....##.#.#.#####.#...###.#..#.###.#....#..#..#.###...#..#.#...#....#.##..##.#
###.....#.####...##.###.#..#.#.##.#.#....#.###...#....#..#...#..##..###.#.#...###.##.#..#...#..###.#...##..######.......##..#.##
#.##..#######..#.##..#..#......##.#####.##....#.##.#.####..##.##.#.#.###.#####..#......#....#.#..##.####..##...#.##..#.#.....#..
.....#.#####.#.####...#####.#...###.#.##..#######......##...###.......#.....###...#.#...#.#..###.

## Day 15: Dueling Generators

Here, you encounter a pair of dueling generators. The generators, called generator A and generator B, are trying to agree on a sequence of numbers. However, one of them is malfunctioning, and so the sequences don't always match.

As they do this, a judge waits for each of them to generate its next value, compares the lowest 16 bits of both values, and keeps track of the number of times those parts of the values match.

The generators both work on the same principle. To create its next value, a generator will take the previous value it produced, multiply it by a factor (generator A uses 16807; generator B uses 48271), and then keep the remainder of dividing that resulting product by 2147483647. That final remainder is the value it produces next.

To calculate each generator's first value, it instead uses a specific starting value as its "previous value" (as listed in your puzzle input).

For example, suppose that for starting values, generator A uses 65, while generator B uses 8921. Then, the first five pairs of generated values are:

--Gen. A--  --Gen. B--
   1092455   430625591
1181022009  1233683848
 245556042  1431495498
1744312007   137874439
1352636452   285222916
In binary, these pairs are (with generator A's value first in each pair):

00000000000100001010101101100111
00011001101010101101001100110111

01000110011001001111011100111001
01001001100010001000010110001000

00001110101000101110001101001010
01010101010100101110001101001010

01100111111110000001011011000111
00001000001101111100110000000111

01010000100111111001100000100100
00010001000000000010100000000100
Here, you can see that the lowest (here, rightmost) 16 bits of the third value match: 1110001101001010. Because of this one match, after processing these five pairs, the judge would have added only 1 to its total.

To get a significant sample, the judge would like to consider 40 million pairs. (In the example above, the judge would eventually find a total of 588 pairs that match in their lowest 16 bits.)

After 40 million pairs, what is the judge's final count?

Your puzzle answer was 569.

--- Part Two ---
In the interest of trying to align a little better, the generators get more picky about the numbers they actually give to the judge.

They still generate values in the same way, but now they only hand a value to the judge when it meets their criteria:

Generator A looks for values that are multiples of 4.
Generator B looks for values that are multiples of 8.
Each generator functions completely independently: they both go through values entirely on their own, only occasionally handing an acceptable value to the judge, and otherwise working through the same sequence of values as before until they find one.

The judge still waits for each generator to provide it with a value before comparing them (using the same comparison method as before). It keeps track of the order it receives values; the first values from each generator are compared, then the second values from each generator, then the third values, and so on.

Using the example starting values given above, the generators now produce the following first five values each:

--Gen. A--  --Gen. B--
1352636452  1233683848
1992081072   862516352
 530830436  1159784568
1980017072  1616057672
 740335192   412269392
These values have the following corresponding binary values:

01010000100111111001100000100100
01001001100010001000010110001000

01110110101111001011111010110000
00110011011010001111010010000000

00011111101000111101010001100100
01000101001000001110100001111000

01110110000001001010100110110000
01100000010100110001010101001000

00101100001000001001111001011000
00011000100100101011101101010000
Unfortunately, even though this change makes more bits similar on average, none of these values' lowest 16 bits match. Now, it's not until the 1056th pair that the judge finds the first match:

--Gen. A--  --Gen. B--
1023762912   896885216

00111101000001010110000111100000
00110101011101010110000111100000
This change makes the generators much slower, and the judge is getting impatient; it is now only willing to consider 5 million pairs. (Using the values from the example above, after five million pairs, the judge would eventually find a total of 309 pairs that match in their lowest 16 bits.)

After 5 million pairs, but using this new generator logic, what is the judge's final count?

Your puzzle answer was 298.

In [235]:
day15in = tuple(get_numbers(x)[0] for x in get_input(15,2017))#(116,299)
#day15in = (116,299)
day15test = (65, 8921)

def doday15(d15in,count, g1 = 1, g2 = 1):
    
    factors = (16807, 48271)
    matches = 0
    divisor = 2147483647
    current = d15in
    for x in range(count):
        if x % 100000 == 0:
            sys.stdout.write('\r%d/%d' %(x,count))
            sys.stdout.flush()
        a = (current[0] * factors[0]) % divisor
        while a % g1 != 0:
            a = (a * factors[0]) % divisor
        b = (current[1] * factors[1]) % divisor
        while b % g2 != 0:
            b = (b * factors[1]) % divisor
        current = (a,b)
        if (current[0] & 0xFFFF) == (current[1] & 0xFFFF):
            matches += 1
    print()
    return matches
        
#assert(doday15(day15test,40000000) == 588)    
print("Part 1:",doday15(day15in,40000000))
#assert(doday15(day15test,5000000,4,8) == 309)    
print("Part 2:",doday15(day15in,5000000,4,8))

Data for day 15 not available, downloading...
39900000/40000000
Part 1: 569
4900000/5000000
Part 2: 298


## Day 16: Permutation Promenade

You come upon a very unusual sight; a group of programs here appear to be dancing.

There are sixteen programs in total, named a through p. They start by standing in a line: a stands in position 0, b stands in position 1, and so on until p, which stands in position 15.

The programs' dance consists of a sequence of dance moves:

Spin, written sX, makes X programs move from the end to the front, but maintain their order otherwise. (For example, s3 on abcde produces cdeab).
Exchange, written xA/B, makes the programs at positions A and B swap places.
Partner, written pA/B, makes the programs named A and B swap places.
For example, with only five programs standing in a line (abcde), they could do the following dance:

s1, a spin of size 1: eabcd.
x3/4, swapping the last two programs: eabdc.
pe/b, swapping programs e and b: baedc.
After finishing their dance, the programs end up in order baedc.

You watch the dance for a while and record their dance moves (your puzzle input). In what order are the programs standing after their dance?

Your puzzle answer was bkgcdefiholnpmja.

--- Part Two ---
Now that you're starting to get a feel for the dance moves, you turn your attention to the dance as a whole.

Keeping the positions they ended up in from their previous dance, the programs perform it again and again: including the first dance, a total of one billion (1000000000) times.

In the example above, their second dance would begin with the order baedc, and use the same dance moves:

s1, a spin of size 1: cbaed.
x3/4, swapping the last two programs: cbade.
pe/b, swapping programs e and b: ceadb.
In what order are the programs standing after their billion dances?

Your puzzle answer was knmdfoijcbpghlea.


In [236]:
day16in = get_input(16,2017)[0].split(',')

def d16(d16in,runcount = 1):
    programs = [x for x in 'abcdefghijklmnop']
    seen = []
    for cnt in range(runcount):
        seen += [''.join(programs)]
        for instruction in d16in:
            #print (''.join(programs),instruction,end=' ')
            tokens = re.findall(r'^[a-z]|\w+',instruction)
            if tokens[0] =='s':
                start = len(programs) - int(tokens[1])
                programs = programs[start:] + programs[:start]
            if tokens[0] == 'x':
                programs[int(tokens[1])] , programs[int(tokens[2])] = programs[int(tokens[2])] , programs[int(tokens[1])]
            if tokens[0] == 'p':
                a = programs.index(tokens[1])
                b = programs.index(tokens[2])
                programs[a] , programs[b] = programs[b] , programs[a]
        if ''.join(programs) in seen:
            return seen[runcount % len(seen)]    
    return ''.join(programs)

print(d16(day16in))
print(d16(day16in,1000000000))

Data for day 16 not available, downloading...
bkgcdefiholnpmja
knmdfoijcbpghlea


## Day 17: Spinlock

Suddenly, whirling in the distance, you notice what looks like a massive, pixelated hurricane: a deadly spinlock. This spinlock isn't just consuming computing power, but memory, too; vast, digital mountains are being ripped from the ground and consumed by the vortex.

If you don't move quickly, fixing that printer will be the least of your problems.

This spinlock's algorithm is simple but efficient, quickly consuming everything in its path. It starts with a circular buffer containing only the value 0, which it marks as the current position. It then steps forward through the circular buffer some number of steps (your puzzle input) before inserting the first new value, 1, after the value it stopped on. The inserted value becomes the current position. Then, it steps forward from there the same number of steps, and wherever it stops, inserts after it the second new value, 2, and uses that as the new current position again.

It repeats this process of stepping forward, inserting a new value, and using the location of the inserted value as the new current position a total of 2017 times, inserting 2017 as its final operation, and ending with a total of 2018 values (including 0) in the circular buffer.

For example, if the spinlock were to step 3 times per insert, the circular buffer would begin to evolve like this (using parentheses to mark the current position after each iteration of the algorithm):

(0), the initial state before any insertions.
0 (1): the spinlock steps forward three times (0, 0, 0), and then inserts the first value, 1, after it. 1 becomes the current position.
0 (2) 1: the spinlock steps forward three times (0, 1, 0), and then inserts the second value, 2, after it. 2 becomes the current position.
0  2 (3) 1: the spinlock steps forward three times (1, 0, 2), and then inserts the third value, 3, after it. 3 becomes the current position.
And so on:

0  2 (4) 3  1
0 (5) 2  4  3  1
0  5  2  4  3 (6) 1
0  5 (7) 2  4  3  6  1
0  5  7  2  4  3 (8) 6  1
0 (9) 5  7  2  4  3  8  6  1
Eventually, after 2017 insertions, the section of the circular buffer near the last insertion looks like this:

1512  1134  151 (2017) 638  1513  851
Perhaps, if you can identify the value that will ultimately be after the last value written (2017), you can short-circuit the spinlock. In this example, that would be 638.

What is the value after 2017 in your completed circular buffer?

Your puzzle answer was 1642.

--- Part Two ---
The spinlock does not short-circuit. Instead, it gets more angry. At least, you assume that's what happened; it's spinning significantly faster than it was a moment ago.

You have good news and bad news.

The good news is that you have improved calculations for how to stop the spinlock. They indicate that you actually need to identify the value after 0 in the current state of the circular buffer.

The bad news is that while you were determining this, the spinlock has just finished inserting its fifty millionth value (50000000).

What is the value after 0 the moment 50000000 is inserted?

Your puzzle answer was 33601318.

In [237]:
day17in = int(get_input(17,2017)[0])


#these could be combined to fill the buffer until your search item is in it,
#then just find things that land on that value...you don't actually need the list
def d171(d17in,maxvalue = 2017, searchafter = 2017):
    pos = 0
    buf = [0]
    for step in range(1,maxvalue + 1):        
        pos = ((pos + d17in) % len(buf)) + 1
        buf.insert(pos,step)
    v = buf.index(searchafter)
    return buf[(v + 1) % len(buf)]

def d172(d17in,maxvalue = 2017, searchafter = 2017):
    pos = 0
    last = 0
    for step in range(1,maxvalue + 1):        
        pos = ((pos + d17in) % step) + 1
        if pos == 1:
            last = step
    return last


print("D17 P1 Test:",d171(3))
print("D17 P1:", d171(day17in))
#print("BIG TEST:", d17(day17in,1000000,0))
print("D17 P2:", d172(day17in,50000000,0))

Data for day 17 not available, downloading...
D17 P1 Test: 638
D17 P1: 1642
D17 P2: 33601318


## Day 18: Duet

You discover a tablet containing some strange assembly code labeled simply "Duet". Rather than bother the sound card with it, you decide to run the code yourself. Unfortunately, you don't see any documentation, so you're left to figure out what the instructions mean on your own.

It seems like the assembly is meant to operate on a set of registers that are each named with a single letter and that can each hold a single integer. You suppose each register should start with a value of 0.

There aren't that many instructions, so it shouldn't be hard to figure out what they do. Here's what you determine:
```
snd X plays a sound with a frequency equal to the value of X.
set X Y sets register X to the value of Y.
add X Y increases register X by the value of Y.
mul X Y sets register X to the result of multiplying the value contained in register X by the value of Y.
mod X Y sets register X to the remainder of dividing the value contained in register X by the value of Y (that is, it sets X to the result of X modulo Y).
rcv X recovers the frequency of the last sound played, but only when the value of X is not zero. (If it is zero, the command does nothing.)
jgz X Y jumps with an offset of the value of Y, but only if the value of X is greater than zero. (An offset of 2 skips the next instruction, an offset of -1 jumps to the previous instruction, and so on.)
```

Many of the instructions can take either a register (a single letter) or a number. The value of a register is the integer it contains; the value of a number is that number.

After each jump instruction, the program continues with the instruction to which the jump jumped. After any other instruction, the program continues with the next instruction. Continuing (or jumping) off either end of the program terminates it.

For example:
```
set a 1
add a 2
mul a a
mod a 5
snd a
set a 0
rcv a
jgz a -1
set a 1
jgz a -2
```
The first four instructions set a to 1, add 2 to it, square it, and then set it to itself modulo 5, resulting in a value of 4.
Then, a sound with frequency 4 (the value of a) is played.
After that, a is set to 0, causing the subsequent rcv and jgz instructions to both be skipped (rcv because a is 0, and jgz because a is not greater than 0).
Finally, a is set to 1, causing the next jgz instruction to activate, jumping back two instructions to another jump, which jumps again to the rcv, which ultimately triggers the recover operation.
At the time the recover operation is executed, the frequency of the last sound played is 4.

What is the value of the recovered frequency (the value of the most recently played sound) the first time a rcv instruction is executed with a non-zero value?

Your puzzle answer was 4601.

--- Part Two ---
As you congratulate yourself for a job well done, you notice that the documentation has been on the back of the tablet this entire time. While you actually got most of the instructions correct, there are a few key differences. This assembly code isn't about sound at all - it's meant to be run twice at the same time.

Each running copy of the program has its own set of registers and follows the code independently - in fact, the programs don't even necessarily run at the same speed. To coordinate, they use the send (snd) and receive (rcv) instructions:

snd X sends the value of X to the other program. These values wait in a queue until that program is ready to receive them. Each program has its own message queue, so a program can never receive a message it sent.
rcv X receives the next value and stores it in register X. If no values are in the queue, the program waits for a value to be sent to it. Programs do not continue to the next instruction until they have received a value. Values are received in the order they are sent.
Each program also has its own program ID (one 0 and the other 1); the register p should begin with this value.

For example:
```
snd 1
snd 2
snd p
rcv a
rcv b
rcv c
rcv d
```
Both programs begin by sending three values to the other. Program 0 sends 1, 2, 0; program 1 sends 1, 2, 1. Then, each program receives a value (both 1) and stores it in a, receives another value (both 2) and stores it in b, and then each receives the program ID of the other program (program 0 receives 1; program 1 receives 0) and stores it in c. Each program now sees a different value in its own copy of register c.

Finally, both programs try to rcv a fourth time, but no data is waiting for either of them, and they reach a deadlock. When this happens, both programs terminate.

It should be noted that it would be equally valid for the programs to run at different speeds; for example, program 0 might have sent all three values and then stopped at the first rcv before program 1 executed even its first instruction.

Once both of your programs have terminated (regardless of what caused them to do so), how many times did program 1 send a value?

Your puzzle answer was 6858.

In [238]:
day18in = get_input(18,2017)

class d18computer:
    def __init__(self, program = [], program_id = 0):
        self.pc = 0
        self.program = program.copy()
        self.registers = dict({(k,0) for k in 'abcdefghijklmnopqrstuvwxyz'})
        self.registers['p'] = program_id
        self.send_queue = []
        self.send_cnt = 0
        self.state = 'RUNNING'
        self.name = 'Program {}'.format(program_id)
        self.other_program = None
        #STATES: 'RUNNING', 'HALTED', 'WAITING'
    
    def get_message_from_queue(self):
        value = self.send_queue[0]
        self.send_queue = self.send_queue[1:]
        self.send_cnt += 1
        return value
        
    def has_message_in_queue(self):
        return len(self.send_queue) > 0
    
    def status(self):
        if self.state == 'WAITING' and self.other_program and self.other_program.has_message_in_queue():
            return 'RUNNING'
        return self.state
    
    def send_count(self):
        return self.send_cnt
    
    def get_value(self, value):
        if value in self.registers:
            return self.registers[value]
        return int(value)
    
    def run(self):
        
        while self.pc >= 0 and self.pc < len(self.program):
            tokens = self.program[self.pc].split(' ')
            
            if tokens[0] == 'snd':
                self.send_queue += [self.get_value(tokens[1])]
                self.pc += 1
                continue
            
            elif tokens[0] == 'set':
                self.registers[tokens[1]] = self.get_value(tokens[2])   
                self.pc += 1
                continue
            elif tokens[0] == 'add':
                self.registers[tokens[1]] += self.get_value(tokens[2])   
                self.pc += 1
                continue
            elif tokens[0] == 'mul':
                self.registers[tokens[1]] *= self.get_value(tokens[2])   
                self.pc += 1
                continue
            elif tokens[0] == 'mod':
                self.registers[tokens[1]] = self.get_value(tokens[1]) % self.get_value(tokens[2])   
                self.pc += 1
                continue
            elif tokens[0] == 'rcv':
                if self.other_program and self.other_program.has_message_in_queue():
                    self.registers[tokens[1]] = self.other_program.get_message_from_queue()
                    self.pc += 1
                else:
                    self.state = 'WAITING'
                    return
            
            elif tokens[0] == 'jgz':
                if self.get_value(tokens[1]) > 0:
                    self.pc += self.get_value(tokens[2])
                    continue 
                self.pc += 1
                continue
        self.state = 'HALTED'
        return
        
    
def d18(d18in):
    pc = 0
    registers = dict({(k,0) for k in 'abcdefghijklmnopqrstuvwxyz'})
    snd = 0
    while pc >= 0 and pc < len(d18in):
        tokens = d18in[pc].split(' ')
        #print(d18in[pc])
        if tokens[0] == 'snd':
            if tokens[1] in registers:
                snd = registers[tokens[1]]
            else:
                snd = int(tokens[1])
        elif tokens[0] == 'set':
            if tokens[2] in registers:
                registers[tokens[1]] = registers[tokens[2]]
            else:
                registers[tokens[1]] = int(tokens[2])
        elif tokens[0] == 'add':
            if tokens[2] in registers:
                registers[tokens[1]] += registers[tokens[2]]
            else:
                registers[tokens[1]] += int(tokens[2])
        elif tokens[0] == 'mul':
            if tokens[2] in registers:
                registers[tokens[1]] *= registers[tokens[2]]
            else:
                registers[tokens[1]] *= int(tokens[2])
        elif tokens[0] == 'mod':
            if tokens[2] in registers:
                registers[tokens[1]] = registers[tokens[1]] % registers[tokens[2]]
            else:
                registers[tokens[1]] = registers[tokens[1]] % int(tokens[2])
            
        elif tokens[0] == 'rcv':
            if registers[tokens[1]] != 0:
                return snd
            pass
        elif tokens[0] == 'jgz':
            if tokens[1] in registers:
                v = registers[tokens[1]]
            else:
                v = int(tokens[1])
            if v > 0:
                if tokens[2] in registers:
                    pc += registers[tokens[2]]
                else:
                    pc += int(tokens[2])
                continue
        pc += 1
        
        
def d182(d18in):
    c0 = d18computer(d18in,0)
    c1 = d18computer(d18in,1)
    
    c0.other_program = c1
    c1.other_program = c0
    
    while c0.status() == 'RUNNING' or c1.status() == 'RUNNING':
        c0.run()
        c1.run()
    return c1.send_count()
    

print('Part 1:',d18(day18in))
print('\rPart 2:',d182(day18in),'                    ')


Data for day 18 not available, downloading...
Part 1: 4601
Part 2: 6858                     


## Day 19: A Series of Tubes

Somehow, a network packet got lost and ended up here. It's trying to follow a routing diagram (your puzzle input), but it's confused about where to go.

Its starting point is just off the top of the diagram. Lines (drawn with |, -, and +) show the path it needs to take, starting by going down onto the only line connected to the top of the diagram. It needs to follow this path until it reaches the end (located somewhere within the diagram) and stop there.

Sometimes, the lines cross over each other; in these cases, it needs to continue going the same direction, and only turn left or right when there's no other option. In addition, someone has left letters on the line; these also don't change its direction, but it can use them to keep track of where it's been. For example:
```
     |          
     |  +--+    
     A  |  C    
 F---|----E|--+ 
     |  |  |  D 
     +B-+  +--+ 
```
Given this diagram, the packet needs to take the following path:

Starting at the only line touching the top of the diagram, it must go down, pass through A, and continue onward to the first +.
Travel right, up, and right, passing through B in the process.
Continue down (collecting C), right, and up (collecting D).
Finally, go all the way left through E and stopping at F.
Following the path to the end, the letters it sees on its path are ABCDEF.

The little packet looks up at you, hoping you can help it find the way. What letters will it see (in the order it would see them) if it follows the path? (The routing diagram is very wide; make sure you view it without line wrapping.)

Your puzzle answer was MKXOIHZNBL.

--- Part Two ---
The packet is curious how many steps it needs to go.

For example, using the same routing diagram from the example above...
```
     |          
     |  +--+    
     A  |  C    
 F---|--|-E---+ 
     |  |  |  D 
     +B-+  +--+ 
```
...the packet would go:

6 steps down (including the first line at the top of the diagram).
3 steps right.
4 steps up.
3 steps right.
4 steps down.
3 steps right.
2 steps up.
13 steps left (including the F it stops on).
This would result in a total of 38 steps.

How many steps does the packet need to go?

Your puzzle answer was 17872.

In [239]:
day19in = get_input(19,2017)

def make_d19_map(map_text):
    the_map = dict()
    for y, line in enumerate(map_text):
        for x, v in enumerate(line):
            the_map[(x,y)] = v
    return the_map

def d19(d19in):
    def find_start(grid):
        i = 0
        while (i,0) in grid:
            if grid[(i,0)] == '|':
                return (i,0)
            i += 1
        return (-1,-1)
    
    def next_step(current, direction):
        return neighbors4(current)[['RIGHT','LEFT','DOWN','UP'].index(direction)]
    
    def find_next_direction(grid, current, last_dir):
        potentials = neighbors4(current)
        if potentials[0] in grid and grid[potentials[0]] in '-+|' and last_dir != 'LEFT':
            return 'RIGHT'
        if potentials[1] in grid and grid[potentials[1]] in '-+|' and last_dir != 'RIGHT':
            return 'LEFT'
        if potentials[2] in grid and grid[potentials[2]] in '|+-' and last_dir != 'UP':
            return 'DOWN'
        if potentials[3] in grid and grid[potentials[3]] in '|+-' and last_dir != 'DOWN':
            return 'UP'
        return direction
    
    direction = 'DOWN'
    current = [find_start(d19in)]
    
    letter_path = []
    
    at_end = False
    while d19in[current[-1]] != ' ':
        while d19in[current[-1]] not in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890+':
            current += [next_step(current[-1], direction)]
        if d19in[current[-1]] != '+':
            letter_path += d19in[current[-1]]
            
        direction = find_next_direction(d19in, current[-1], direction)
        current += [next_step(current[-1], direction)]
    current.pop() #get rid of the ' ' at the end    
    return ''.join(letter_path),len(current)

d19map = make_d19_map(day19in)

print('Part 1:',underline(bold(str(d19(d19map)[0]))))
print('Part 2:',underline(bold(str(d19(d19map)[1]))))



Data for day 19 not available, downloading...
Part 1: MKXOIHZNBL
Part 2: 17872


## Day 20: Particle Swarm

Suddenly, the GPU contacts you, asking for help. Someone has asked it to simulate too many particles, and it won't be able to finish them all in time to render the next frame at this rate.

It transmits to you a buffer (your puzzle input) listing each particle in order (starting with particle 0, then particle 1, particle 2, and so on). For each particle, it provides the X, Y, and Z coordinates for the particle's position (p), velocity (v), and acceleration (a), each in the format <X,Y,Z>.

Each tick, all particles are updated simultaneously. A particle's properties are updated in the following order:
```
Increase the X velocity by the X acceleration.
Increase the Y velocity by the Y acceleration.
Increase the Z velocity by the Z acceleration.
Increase the X position by the X velocity.
Increase the Y position by the Y velocity.
Increase the Z position by the Z velocity.
```
Because of seemingly tenuous rationale involving z-buffering, the GPU would like to know which particle will stay closest to position <0,0,0> in the long term. Measure this using the Manhattan distance, which in this situation is simply the sum of the absolute values of a particle's X, Y, and Z position.

For example, suppose you are only given two particles, both of which stay entirely on the X-axis (for simplicity). Drawing the current states of particles 0 and 1 (in that order) with an adjacent a number line and diagram of current X positions (marked in parenthesis), the following would take place:
```
p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>                         (0)(1)

p=< 4,0,0>, v=< 1,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=< 2,0,0>, v=<-2,0,0>, a=<-2,0,0>                      (1)   (0)

p=< 4,0,0>, v=< 0,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=<-2,0,0>, v=<-4,0,0>, a=<-2,0,0>          (1)               (0)

p=< 3,0,0>, v=<-1,0,0>, a=<-1,0,0>    -4 -3 -2 -1  0  1  2  3  4
p=<-8,0,0>, v=<-6,0,0>, a=<-2,0,0>                         (0)   
```
At this point, particle 1 will never be closer to <0,0,0> than particle 0, and so, in the long run, particle 0 will stay closest.

Which particle will stay closest to position <0,0,0> in the long term?

Your puzzle answer was 119.

--- Part Two ---
To simplify the problem further, the GPU would like to remove any particles that collide. Particles collide if their positions ever exactly match. Because particles are updated simultaneously, more than two particles can collide at the same time and place. Once particles collide, they are removed and cannot collide with anything else after that tick.

For example:
```
p=<-6,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-4,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=<-2,0,0>, v=< 1,0,0>, a=< 0,0,0>    (0)   (1)   (2)            (3)
p=< 3,0,0>, v=<-1,0,0>, a=< 0,0,0>

p=<-3,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-2,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=<-1,0,0>, v=< 1,0,0>, a=< 0,0,0>             (0)(1)(2)      (3)   
p=< 2,0,0>, v=<-1,0,0>, a=< 0,0,0>

p=< 0,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=< 0,0,0>, v=< 2,0,0>, a=< 0,0,0>    -6 -5 -4 -3 -2 -1  0  1  2  3
p=< 0,0,0>, v=< 1,0,0>, a=< 0,0,0>                       X (3)      
p=< 1,0,0>, v=<-1,0,0>, a=< 0,0,0>

------destroyed by collision------    
------destroyed by collision------    -6 -5 -4 -3 -2 -1  0  1  2  3
------destroyed by collision------                      (3)         
p=< 0,0,0>, v=<-1,0,0>, a=< 0,0,0>
```
In this example, particles 0, 1, and 2 are simultaneously destroyed at the time and place marked X. On the next tick, particle 3 passes through unharmed.

How many particles are left after all collisions are resolved?

Your puzzle answer was 471.

In [296]:
day20input = get_input(20,2017)

class v3:
    def __init__(self, numbers):
        self.p = numbers[0:3]
        self.v = numbers[3:6]
        self.a = numbers[6:9]
        
    def update(self):
        self.v[0] += self.a[0]
        self.v[1] += self.a[1]
        self.v[2] += self.a[2]
        self.p[0] += self.v[0]
        self.p[1] += self.v[1]
        self.p[2] += self.v[2]
        
    def __repr__(self):
        return f'p={self.p} v={self.v} a={self.a}'
    
    def __eq__(self, other):
        return self.p[0] == other.p[0] and self.p[1] == other.p[1] and self.p[2] == other.p[2]
    
    def __hash__(self):
        return hash(self.p[0]) ^ hash(self.p[1]) ^ hash(self.p[2])
    
    def update_to_time(self, t):
        sum_t = (t * (t + 1)) // 2
        self.p = [self.p[0] + (t) * self.v[0] + (sum_t) * self.a[0], self.p[1] + (t) * self.v[1] + (sum_t) * self.a[1], self.p[2] + (t) * self.v[2] + (sum_t) * self.a[2]]
        self.v = [self.v[0] + (t) * self.a[0], self.v[1] + (t) * self.a[1], self.v[2] + (t) * self.a[2]]
    
    def collides_with(self,o):
        solutions = []
        for i in range(3):
            sol = quadratic_solutions((o.a[i]-self.a[i]),2*(o.v[i] - self.v[i] + o.a[i] - self.a[i]),2 * (o.p[i] - self.p[i]))
            if not sol or len(sol) != 2 or (sol[0] < 0 and sol[1]):
                return False
            solutions += [sol]
        try:
            return (solutions[0][0] in solutions[1] and solutions[0][0] in solutions[2]) or (solutions[0][1] in solutions[1] and solutions[0][0] in solutions[2])
        except:
            print(solutions)
            return False
        

    def distance_from_zero(v):
        return abs(v.p[0]) + abs(v.p[1]) + abs(v.p[2])
    
    
def d20(d20in, collisions = False):
    points = []
    idx = 0
    for line in d20in:
        points += [v3(get_numbers(line))]
    
    #this would be a shit ton easier if you just looked at the lowest accelleration, (if tie) lowest velocity, (if tie) lowest position
    if not collisions:
        for p in points:
            p.update_to_time(1000000)
        distances = [distance_from_zero(x) for x in points]    
        return distances.index(min(distances))
    
    #protip, use math..    
    for i in range(500):
        for p in points:
            p.update()
        
        if collisions:
            j = 0
            while j < len(points):
                if points.count(points[j]) > 1:
                    stored = points[j]
                    while points.count(stored) > 0:
                        points.remove(stored)
                else:
                    j+=1
        println(f'run: {i} num_parts: {len(points)}      ')
        
    if collisions:
        return len(points)
    else:
        return idx

print('Part 1:',underline(bold(str(d20(day20input)))))
print('Part 2:',underline(bold(str(d20(day20input, True)))))


Part 1: 119
run: 499 num_parts: 471      Part 2: 471


## Day 21


In [ ]:
day21input = get_input(21, 2017)


class d21_grid:
    def __init__(self, in_string, default='.'):
        self.height = 0
        self.width = 0

        self.grid = defaultdict(lambda: default)
        
        if in_string.count('/') > 0:
            self.stored_string = ''
            for y, line in enumerate(in_string.split('/')):
                for x, char in enumerate(line):
                        self.grid[(x, y)] = char
                        self.stored_string += char

            self.width = len(in_string.split('/')[0])
            self.height = len(in_string.split('/'))
            self.dirty = False
        else:
            i = 0
            self.height = int(math.sqrt(len(in_string)))
            self.width = self.height
            for y in range(self.height):
                for x in range(self.width):
                        self.grid[(x, y)] = in_string[i]
                        i += 1    
            self.dirty = False
            self.stored_string = in_string
        self.clean()

    def __repr__(self):
        return "grid('{}')".format(self.make_grid_string())

    def blit(self, start, new_grid):
        """
        start = (x,y) new_grid = grid
        """

        new_size = (max(self.width, start[0] + new_grid.width),
                    max(self.height, start[1] + new_grid.height))
        
        self.reserve(new_size)
        self.dirty = True
        for k,v in new_grid.grid.items():
            self.grid[(k[0]+start[0],k[1]+start[1])] = v
            
        

    def reserve(self, size):
        if size[1] > self.height:
            self.dirty = True
            self.height = size[1]
        
        if size[0] > self.width:
            self.dirty = True
            self.width = size[0]
        

    def clean(self):

        self.grid = defaultdict(lambda: '.', {k:v for (k,v) in self.grid.items() if v == '#'})


    def rotated(self):
        g = d21_grid(self.make_grid_string())
        for i in range(g.width//2):
            for j in range(i,g.width - i - 1):
                g.grid[(i,j)] , g.grid[(j,g.width-i-1)], g.grid[(g.width-i-1,g.width-j-1)],g.grid[(g.width-j-1,i)] = self.grid[(j,g.width-i-1)], self.grid[(g.width-i-1,g.width-j-1)],self.grid[(g.width-j-1,i)] ,self.grid[(i,j)] 
        g.clean()
        g.dirty = True
        return g

    def flipped_h(self):
        g = d21_grid(self.make_grid_string())
        for i in range(g.width):
            g.grid[(0, i)], g.grid[(g.width - 1, i)] = self.grid[(g.width - 1, i)], self.grid[(0, i)]
        g.dirty = True
        g.clean()
        return g

    def flipped_v(self):
        g = d21_grid(self.make_grid_string())
        for i in range(g.width):
            g.grid[(i, 0)], g.grid[(i, g.width - 1)
                                   ] = g.grid[(i, g.width - 1)], g.grid[(i, 0)]
        g.dirty = True
        g.clean()
        return g

    def sub_grid(self, start, size):
        g = d21_grid('')
        g.reserve(size)
        for y in range(size[1]):
            for x in range(size[0]):
                if (x,y) in self.grid:
                    g.grid[(x, y)] = self.grid[(start[0] + x, start[1] + y)]
        g.dirty = True
        return g

    def split(self, size):
        grids = []
        for y in range(0, self.height, size[1]):
            for x in range(0, self.width, size[0]):
                grids += [self.sub_grid((x, y), size)]
        return grids

    def __hash__(self):
        return hash(self.make_grid_string())

    def __eq__(self, other):
        return self.width == other.width and self.height == other.height and self.make_grid_string() == other.make_grid_string()

    def make_grid_string(self):
        if self.dirty:
            self.stored_string = ''
            for y in range(self.height):
                for x in range(self.width):
                    self.stored_string += self.grid[(x, y)]
            self.dirty = False
        return self.stored_string

    # only works for square grids
    def grid_from_combining_grids(grids):
        rg = d21_grid('')
        grids_wide = int(math.sqrt(len(grids)))
        step = grids[0].width
        rg.reserve((grids_wide * step, grids_wide * step))
        rg.make_grid_string()
        gnum = 0
        num_grids = len(grids)
        for y in range(grids_wide):
            for x in range(grids_wide):
                #sys.stdout.write("{}/{} computed\r".format(gnum,num_grids))
                #sys.stdout.flush()
                rg.blit((x * step, y * step), grids[gnum])
                gnum += 1
        rg.dirty = True
        #print()
        return rg


def make_trans(translations, g_in):
    if g_in not in translations:
        if g_in.width % 6 == 0 and g_in.width > 6 and (g_in.width / 6) % 2 == 0:
            cg = g_in.split((6,6))
        elif g_in.width % 2 == 0:
            cg = g_in.split((2,2))
        elif g_in.width % 9 == 0 and g_in.width > 9 and (g_in.width / 9) % 2 != 0 and (g_in.width / 9) % 3 == 0:
            cg = g_in.split((9,9))
        else:
            cg = g_in.split((3,3)) 

        out = d21_grid.grid_from_combining_grids([make_trans(translations,wg) for wg in cg])
        
        if g_in.width < 10:
            translations[g_in] = out

        return out
    return translations[g_in]



def d21(d21in, count=5):
    master = d21_grid('.#./..#/###')

    translations = dict()
    for line in d21in:
        a, b = re.findall('[#./]+', line)
        key = d21_grid(a)
        val = d21_grid(b)

      
        translations[key] = val
        key = key.rotated()
        translations[key] = val
        key = key.rotated()
        translations[key] = val
        key = key.rotated()
        translations[key] = val
        key = key.flipped_v()
        translations[key] = val
        key = key.rotated()
        translations[key] = val
        key = key.rotated()
        translations[key] = val
        key = key.rotated()
        translations[key] = val

    for iter in range(count):
        master = make_trans(translations,master)
        print(iter+1, time.clock())



    return master.make_grid_string().count('#')


d21test = '''../.# => ##./#../...
.#./..#/### => #..#/..../..../#..#'''.splitlines()
#print('Test:', d21(d21test, 2))
print('Part 1:', d21(day21input))
print('Speed Test',d21(day21input,11))
#print('Part 2:',d21(day21input,18))

In [26]:
day22input = get_input(22, 2017)

def print_d22(the_grid, cursor, size):
    pass

def d22_1(d22in,bursts):
    d22grid = defaultdict(lambda: '.')
    height = len(d22in)
    width = len(d22in)
    start_x = -(height//2)
    start_y = -(width//2)
    infector_count = 0
    for y, line in enumerate(d22in):
        for x,char in enumerate(line):
            d22grid[(x+start_x,y+start_y)] = char
    
    directions = ['up','right','down','left']
    carrier = {'pos':(0,0),'dir':'up'}
    for i in range(bursts):
        if d22grid[carrier['pos']] == '#':
            t = (directions.index(carrier['dir']) + 1) % 4
            carrier['dir'] = directions[t]
            d22grid[carrier['pos']] = '.'
        else:
            t = (directions.index(carrier['dir']) - 1)
            carrier['dir'] = directions[t]
            d22grid[carrier['pos']] = '#'
            infector_count += 1
       
        if carrier['dir'] == 'up':
            carrier['pos'] = (carrier['pos'][0],carrier['pos'][1]-1)
        if carrier['dir'] == 'down':
            carrier['pos'] = (carrier['pos'][0],carrier['pos'][1]+1)
        if carrier['dir'] == 'left':
            carrier['pos'] = (carrier['pos'][0]-1,carrier['pos'][1])
        if carrier['dir'] == 'right':
            carrier['pos'] = (carrier['pos'][0]+1,carrier['pos'][1])
            
    return infector_count

def d22_2(d22in,bursts):
    d22grid = defaultdict(lambda: '.')
    height = len(d22in)
    width = len(d22in)
    start_x = -(height//2)
    start_y = -(width//2)
    infector_count = 0
    for y, line in enumerate(d22in):
        for x,char in enumerate(line):
            d22grid[(x+start_x,y+start_y)] = char
    
    directions = ['up','right','down','left']
    carrier = {'pos':(0,0),'dir':'up'}
    for i in range(bursts):
        if d22grid[carrier['pos']] == '#':
            t = (directions.index(carrier['dir']) + 1) % 4
            carrier['dir'] = directions[t]
            d22grid[carrier['pos']] = 'F'
        elif d22grid[carrier['pos']] == '.':
            t = (directions.index(carrier['dir']) - 1)
            carrier['dir'] = directions[t]
            d22grid[carrier['pos']] = 'W'
        elif d22grid[carrier['pos']] == 'F':
            t = (directions.index(carrier['dir']) +2 ) % 4
            carrier['dir'] = directions[t]
            d22grid[carrier['pos']] = '.'
        elif d22grid[carrier['pos']] == 'W':
            infector_count += 1
            d22grid[carrier['pos']] = '#'
       
        if carrier['dir'] == 'up':
            carrier['pos'] = (carrier['pos'][0],carrier['pos'][1]-1)
        if carrier['dir'] == 'down':
            carrier['pos'] = (carrier['pos'][0],carrier['pos'][1]+1)
        if carrier['dir'] == 'left':
            carrier['pos'] = (carrier['pos'][0]-1,carrier['pos'][1])
        if carrier['dir'] == 'right':
            carrier['pos'] = (carrier['pos'][0]+1,carrier['pos'][1])
            
    return infector_count

d22test = '''..#
#..
...'''.splitlines()
print('Test 1',d22_1(d22test,10000))
print('Part 1:',d22_1(day22input,10000))
print('Test 2:',d22_2(d22test,10000000))
print('Part 2:',d22_2(day22input,10000000))

Test 1 5587
Part 1: 5450
Test 2: 2511944
Part 2: 2511957


'''
set b 84
set c b
jnz a start_new
jnz 1 start
start_new: mul b 100
sub b -100000
set c b
sub c -17000
start: set f 1
set d 2
loop2: set e 2
loop1: set g d
mul g e
sub g b
jnz g skip1
set f 0
skip1: sub e -1
set g e
sub g b
jnz g loop1
sub d -1
set g d
sub g b
jnz g loop2
jnz f skip1
sub h -1
skip2: set g b
sub g c
jnz g skip3
jnz 1 end
skip3: sub b -17
jnz 1 start
end: HALT




int test()
{
	cnt = 0
	for(b = 108400; b <= 125400; b+=17)
	{ 
		f = 1
		d = 2
		
		for(d = 2; d <= b; d++)
		{
			for(e = 2; e <= b; e++)
			{
				g = (d * e) - b // this is a prime test, if any 2 numbers multiply to b, increase a counter,
				if(g==0)
				{
					f = 0
					goto out;
				}
			}
		}
		
		if(f==0)
		{
			cnt++
		}
	}
	return cnt
end: HALT

b = 108400
c = 125400
while(true)
{
	do{
		f = 1
		d = 2
		e = 2
		do{
			g = (d*e)-b
			if (g == 0)
			{
				f = 0
			}
			e++
			g = e - b
		}while(g != 0)
		d++
		g = d
		g = g - b
	}while(g!=0)
	if f==0
	{
		return
	}
	g = b
	g = g - c
	if g == 0
	{
		return
	}
	b = b + 17
}
'''

In [42]:
day23input = get_input(23, 2017)



def d23(d23in, part2 = False):
    
    def reg_or_val(val):
        try:
            return int(val)
        except:
            return registers[val]
    
    registers = {k:0 for k in 'abcdefgh'}
    
    def reg_or_val(val):
        try:
            return int(val)
        except:
            return registers[val]
    cycles = 0
    instruction_count = {k:0 for k in ['mul','set','sub','jnz']}
    
    p2 = 0
    if part2:
        #it's calculating the numbers of non-primes between 108400 and 125401 (skipping every 17 numbers)
        primes = list(primes_under(125401))
        p2 = len([x for x in list(range(108400,125401,17)) if x not in primes])
        
    
    pc = 0
    while pc >=0  and pc < len(d23in) and cycles < 1000000:
        cycles += 1
        instruction = d23in[pc].split()
        instruction_count[instruction[0]] += 1
        if instruction[0] == 'set':
            pc += 1
            registers[instruction[1]] = reg_or_val(instruction[2])
        if instruction[0] == 'sub':
            pc += 1
            registers[instruction[1]] -= reg_or_val(instruction[2])
        if instruction[0] == 'mul':
            pc += 1
            registers[instruction[1]] *= reg_or_val(instruction[2])
        if instruction[0] == 'jnz':
            if reg_or_val(instruction[1]) != 0:
                pc += reg_or_val(instruction[2])
            else:
                pc += 1
    
    
    return instruction_count['mul'],p2

print('Part 1:',d23(day23input)[0])
print('Part 2:',d23(day23input,True)[1])

Part 1: 6724
Part 2: 903


In [63]:
day24input = get_input(24, 2017)
import copy
def d24(d24in,part2 = False):
    components = []
    for line in d24in:
        components.append(tuple(get_numbers(line))) 
    
    path = [(0,0)]
    avalable_paths = 0
    def find_max_path_from(connect_with, current_path, remaining):
        values = []
        
        for connection in remaining:
            if connection[0] == connect_with:
                next_round = copy.deepcopy(remaining)
                next_round.remove(connection)
                values += [find_max_path_from(connection[1],current_path + [connection],next_round)]
            elif connection[1] == connect_with:
                next_round = copy.deepcopy(remaining)
                next_round.remove(connection)
                values += [find_max_path_from(connection[0],current_path + [connection],next_round)]
        
        if len(values) == 0:
            return sum([v[0] + v[1] for v in current_path])
        return max(values)
    
    
    def find_max_path(connect_with, current_path, remaining):
        max_path = []
        
        for connection in remaining:
            if connection[0] == connect_with:
                next_round = copy.deepcopy(remaining)
                next_round.remove(connection)
                potential_max = find_max_path(connection[1],current_path + [connection],next_round)
                if len(potential_max) > len(max_path):
                    max_path = potential_max
                if len(potential_max) == len(max_path) and sum([v[0] + v[1] for v in potential_max]) > sum([v[0] + v[1] for v in max_path]):
                    max_path = potential_max
            elif connection[1] == connect_with:
                next_round = copy.deepcopy(remaining)
                next_round.remove(connection)
                potential_max = find_max_path(connection[0],current_path + [connection],next_round)
                if len(potential_max) > len(max_path):
                    max_path = potential_max
                if len(potential_max) == len(max_path) and sum([v[0] + v[1] for v in potential_max]) > sum([v[0] + v[1] for v in max_path]):
                    max_path = potential_max
        
        if len(max_path) == 0:
            return current_path 
        return max_path
    
    if not part2:
        return find_max_path_from(0,path, components)
    else:
        return sum([v[0] + v[1] for v in  find_max_path(0,path,components)])
    

print("Part 1:",d24(day24input))
print("Part 2:",d24(day24input, True))

Part 1: 1511
Part 2: 1471


In [68]:
day25input = get_input(25, 2017)

def d25(d25in):
    tape = defaultdict(int)
    
    states = {'A':({'write':1,'move':1,'next':'B'},{'write':0,'move':-1,'next':'F'}),
              'B':({'write':0,'move':1,'next':'C'},{'write':0,'move':1,'next':'D'}),
              'C':({'write':1,'move':-1,'next':'D'},{'write':1,'move':1,'next':'E'}),
              'D':({'write':0,'move':-1,'next':'E'},{'write':0,'move':-1,'next':'D'}),
              'E':({'write':0,'move':1,'next':'A'},{'write':1,'move':1,'next':'C'}),
              'F':({'write':1,'move':-1,'next':'A'},{'write':1,'move':1,'next':'A'})}
    current_state = 'A'
    cursor = 0
    for cnt in range(12794428):
        current_val = tape[cursor]
        tape[cursor] = states[current_state][current_val]['write']
        cursor += states[current_state][current_val]['move']
        current_state = states[current_state][current_val]['next']
    
    
    return sum(list(tape.values()))
print('Part 1:',d25(day25input))

Part 1: 2832
